# Import

In [1]:


from guardrail_dataset import IS_Dataset
from guardrail_dataset_test import IS_Dataset_test

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from PIL import Image
from torch.utils import data
from torchvision import transforms, utils
import torch.nn.utils.prune as prune
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import torchvision
import random
import torch.optim as opt
from tqdm import tqdm
import time
from thop import profile

import os
from torchsummary import summary
import cv2
from sklearn.model_selection import train_test_split

import copy
from collections import defaultdict
from collections import OrderedDict

import re
import math



# from metrics import flops, model_size

In [2]:
torch.cuda.is_available()

True

# Model

## EDAnet

In [127]:
class DownsampleBlock(nn.Module):
    def __init__(self, nc_input, nc_output):
        '''
        Arguments:
        nc_input : Win, number of input channel
        nc_output : Wout, number of output channel
        '''

        super(DownsampleBlock,self).__init__()
        self.nc_input = nc_input
        self.nc_output = nc_output

        if self.nc_input < self.nc_output:
          # Win < Wout
            self.conv = nn.Conv2d(nc_input, nc_output-nc_input, kernel_size=3, stride=2, padding=1)
            self.pool = nn.MaxPool2d(2, stride=2)
        else:
          # Win > Wout
            self.conv = nn.Conv2d(nc_input, nc_output, kernel_size=3, stride=2, padding=1)

        self.batchNorm = nn.BatchNorm2d(nc_output)
        self.relu = nn.ReLU()

    def forward(self, x):
        if self.nc_input < self.nc_output:
            out = torch.cat([self.conv(x), self.pool(x)], 1)
        else:
            out = self.conv(x)

        out = self.batchNorm(out)
        out = self.relu(out)
        return out

class EDABlock(nn.Module):
    def __init__(self, nc_input, dilated, k = 40, dropprob = 0.02):
        '''
        Arguments:
        nc_input : number of input channel
        k : growth rate
        dilated : possible dilated convalution
        dropprob : probability, a dropout layer between the last ReLU and the concatenation of each module
        '''
        super(EDABlock,self).__init__()
        self.conv1x1_0 = nn.Conv2d(nc_input, k, kernel_size=1)
        self.batchNorm_0 = nn.BatchNorm2d(k)

        self.conv3x1_1 = nn.Conv2d(k, k, kernel_size=(3,1), padding=(1,0))
        self.conv1x3_1 = nn.Conv2d(k, k, kernel_size=(1,3), padding=(0,1))
        self.batchNorm_1 = nn.BatchNorm2d(k)

        self.conv3x1_2 = nn.Conv2d(k, k, kernel_size=(3,1), stride=1, padding=(dilated,0), dilation=dilated)
        self.conv1x3_2 = nn.Conv2d(k, k, kernel_size=(1,3), stride=1, padding=(0,dilated), dilation=dilated)
        self.batchNorm_2 = nn.BatchNorm2d(k)
        self.dropout = nn.Dropout2d(dropprob)
        self.relu = nn.ReLU()

    def forward(self, x):
        input = x

        output = self.conv1x1_0(x)
        output = self.batchNorm_0(output)
        output = self.relu(output)

        output = self.conv3x1_1(output)
        output = self.conv1x3_1(output)
        output = self.batchNorm_1(output)
        output = self.relu(output)

        output = self.conv3x1_2(output)
        output = self.conv1x3_2(output)
        output = self.batchNorm_2(output)
        output = self.relu(output)

        if (self.dropout.p != 0):
            output = self.dropout(output)

        output = torch.cat((output, input), 1)

        return output

class EDAnet(nn.Module):
    def __init__(self, n_class=1):
        '''
        Arguments:
        nc_input : number of input channel
        k : growth rate
        dilated : possible dilated convalution
        dropprob : probability, a dropout layer between the last ReLU and the concatenation of each module
        '''
        super(EDAnet,self).__init__()
        self.layers = nn.ModuleList()
        self.dilation1 = [1,1,1,2,2]
        self.dilation2 = [2,2,4,4,8,8,16,16]

        # DownsampleBlock1
        self.layers.append(DownsampleBlock(3, 15))

        # DownsampleBlock2
        self.layers.append(DownsampleBlock(15, 60))

        # EDA module 1-1~1-5
        for i in range(len(self.dilation1)):
            self.layers.append(EDABlock(60 + 40 * i, self.dilation1[i]))

        # DownsampleBlock3
        self.layers.append(DownsampleBlock(260, 130))

        # EDA module 2-1~2-8
        for j in range(len(self.dilation2)):
            self.layers.append(EDABlock(130 + 40 * j, self.dilation2[j]))

        # Projection layer
        self.project_layer = nn.Conv2d(450, n_class, kernel_size = 1)

        self.weights_init()

    def weights_init(self):
        for index, m in enumerate(self.modules()):
            classname = m.__class__.__name__
            if classname.find('Conv') != -1:
                m.weight.data.normal_(0.0, 0.02)
            elif classname.find('BatchNorm') != -1:
                m.weight.data.normal_(1.0, 0.02)
                m.bias.data.fill_(0)

    def forward(self, x):
        output = x

        for layer in self.layers:
            output = layer(output)
          # print(output.shape)

        output = self.project_layer(output)

        # Bilinear interpolation x8
        output = F.interpolate(output,scale_factor = 8,mode = 'bilinear',align_corners=True)

        # # Bilinear interpolation x2 (inference only)
        # if not self.training:
        #   output = F.interpolate(output, scale_factor=2, mode='bilinear',align_corners=True)

        return output

## EDAnet ghost

In [129]:
class GhostModule(nn.Module):
    def __init__(self, inp, oup, kernel_size=1, ratio=2, dw_size=3, stride=1, relu=True, dilated=None):
        super(GhostModule, self).__init__()
        self.oup = oup
        init_channels = math.ceil(oup / ratio)
        new_channels = init_channels*(ratio-1)
        self.primary_conv = nn.Sequential(
            nn.Conv2d(inp, init_channels, kernel_size, stride, kernel_size//2, bias=False),
            nn.BatchNorm2d(init_channels),
            nn.ReLU(inplace=True) if relu else nn.Sequential(),
        )

        self.cheap_operation = nn.Sequential(
            nn.Conv2d(init_channels, new_channels, dw_size, 1, dw_size//2, groups=init_channels, bias=False),
            nn.BatchNorm2d(new_channels),
            nn.ReLU(inplace=True) if relu else nn.Sequential(),
        )

    def forward(self, x):
        x1 = self.primary_conv(x)
        x2 = self.cheap_operation(x1)
        out = torch.cat([x1,x2], dim=1)
        return out[:,:self.oup,:,:]
    
class DownsampleBlock(nn.Module):
  def __init__(self, nc_input, nc_output):
    '''
    Arguments:
    nc_input : Win, number of input channel
    nc_output : Wout, number of output channel
    '''

    super(DownsampleBlock,self).__init__()
    self.nc_input = nc_input
    self.nc_output = nc_output

    if self.nc_input < self.nc_output:
      # Win < Wout
      self.conv = GhostModule(nc_input, nc_output-nc_input, kernel_size=3, stride=2, relu=True)
      # self.conv = nn.Conv2d(nc_input, nc_output-nc_input, kernel_size=3, stride=2, padding=1)
      self.pool = nn.MaxPool2d(2, stride=2)
    else:
      # Win > Wout
      self.conv = GhostModule(nc_input, nc_output, kernel_size=3, stride=2, relu=True)
      # self.conv = nn.Conv2d(nc_input, nc_output, kernel_size=3, stride=2, padding=1)

    self.batchNorm = nn.BatchNorm2d(nc_output)
    self.relu = nn.ReLU()

  def forward(self, x):
    if self.nc_input < self.nc_output:
      out = torch.cat([self.conv(x), self.pool(x)], 1)
    else:
      out = self.conv(x)
    
    # out = self.batchNorm(out)
    # out = self.relu(out)
    return out



class EDABlock_ghost(nn.Module):
  def __init__(self, nc_input, dilated, k = 40, dropprob = 0.02):
    '''
    Arguments:
    nc_input : number of input channel
    k : growth rate
    dilated : possible dilated convalution
    dropprob : probability, a dropout layer between the last ReLU and the concatenation of each module
    '''
    # GhostModule(inp, hidden_dim, kernel_size=1, relu=True)
    super(EDABlock_ghost,self).__init__()
    # self.conv1x1_0 = nn.Conv2d(nc_input, k, kernel_size=1)
    self.conv_0 = GhostModule(nc_input, k, kernel_size=1, relu=True)
    self.batchNorm_0 = nn.BatchNorm2d(k)

    # self.conv3x1_1 = nn.Conv2d(k, k, kernel_size=(3,1), padding=(1,0))
    # self.conv1x3_1 = nn.Conv2d(k, k, kernel_size=(1,3), padding=(0,1))
    self.conv_1 = GhostModule(k, k, kernel_size=3, relu=True)
    self.batchNorm_1 = nn.BatchNorm2d(k)

    self.conv3x1_2 = nn.Conv2d(k, k, kernel_size=(3,1), stride=1, padding=(dilated,0), dilation=dilated)
    self.conv1x3_2 = nn.Conv2d(k, k, kernel_size=(1,3), stride=1, padding=(0,dilated), dilation=dilated)
    self.batchNorm_2 = nn.BatchNorm2d(k)
    self.dropout = nn.Dropout2d(dropprob)
    self.relu = nn.ReLU()

  def forward(self, x):
    input = x

    output = self.conv_0(x)
    # output = self.conv1x1_0(x)
    # output = self.batchNorm_0(output)
    # output = self.relu(output)

    output = self.conv_1(output)
    # output = self.conv3x1_1(output)
    # output = self.conv1x3_1(output)
    # output = self.batchNorm_1(output)
    # output = self.relu(output)

    output = self.conv3x1_2(output)
    output = self.conv1x3_2(output)
    output = self.batchNorm_2(output)
    output = self.relu(output)

    if (self.dropout.p != 0):
      output = self.dropout(output)

    output = torch.cat((output, input), 1)

    return output


class EDAnet_ghost(nn.Module):
  def __init__(self, n_class=1):
    '''
    Arguments:
    nc_input : number of input channel
    k : growth rate
    dilated : possible dilated convalution
    dropprob : probability, a dropout layer between the last ReLU and the concatenation of each module
    '''
    super(EDAnet_ghost,self).__init__()
    self.layers = nn.ModuleList()
    self.dilation1 = [1,1,1,2,2]
    self.dilation2 = [2,2,4,4,8,8,16,16]

    # DownsampleBlock1
    self.layers.append(DownsampleBlock(3, 15))

    # DownsampleBlock2
    self.layers.append(DownsampleBlock(15, 60))

    # EDA module 1-1~1-5
    for i in range(len(self.dilation1)):
      self.layers.append(EDABlock_ghost(60 + 40 * i, self.dilation1[i]))

    # DownsampleBlock3
    self.layers.append(DownsampleBlock(260, 130))

    # EDA module 2-1~2-8
    for j in range(len(self.dilation2)):
      self.layers.append(EDABlock_ghost(130 + 40 * j, self.dilation2[j]))

    # Projection layer
    self.project_layer = nn.Conv2d(450, n_class, kernel_size = 1)

    self.weights_init()
  
  def weights_init(self):
    for index, m in enumerate(self.modules()):
      classname = m.__class__.__name__
      if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
      elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

  def forward(self, x):
    output = x

    for layer in self.layers:
      output = layer(output)
      # print(output.shape)

    output = self.project_layer(output)

    # Bilinear interpolation x8
    output = F.interpolate(output,scale_factor = 8,mode = 'bilinear',align_corners=True)

    # # Bilinear interpolation x2 (inference only)
    # if not self.training:
    #   output = F.interpolate(output, scale_factor=2, mode='bilinear',align_corners=True)

    return output



## Unet++

In [24]:
class conv_block_nested(nn.Module):

    def __init__(self, in_ch, mid_ch, out_ch):
        super(conv_block_nested, self).__init__()
        self.activation = nn.ReLU(inplace=True)
        
        self.conv3x1_1 = nn.Conv2d(in_ch, in_ch, kernel_size=(3,1), padding=(1,0), bias=True)
        self.conv1x3_1 = nn.Conv2d(in_ch, mid_ch, kernel_size=(1,3), padding=(0,1), bias=True)
        
#         self.conv1 = nn.Conv2d(in_ch, mid_ch, kernel_size=3, padding=1, bias=True)
        self.bn1 = nn.BatchNorm2d(mid_ch)
    
        self.conv3x1_2 = nn.Conv2d(mid_ch, mid_ch, kernel_size=(3,1), padding=(1,0), bias=True)
        self.conv1x3_2 = nn.Conv2d(mid_ch, out_ch, kernel_size=(1,3), padding=(0,1), bias=True)
#         self.conv2 = nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1, bias=True)
        self.bn2 = nn.BatchNorm2d(out_ch)

    def forward(self, x):
#         x = self.conv1(x)
        x = self.conv3x1_1(x)
        x = self.conv1x3_1(x)
        
        x = self.bn1(x)
        x = self.activation(x)

#         x = self.conv2(x)
        x = self.conv3x1_2(x)
        x = self.conv1x3_2(x)
        x = self.bn2(x)
        output = self.activation(x)

        return output

class Nested_UNet(nn.Module):

    def __init__(self, in_ch=3, out_ch=1):
        super(Nested_UNet, self).__init__()

        n1 = 32
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = conv_block_nested(in_ch, filters[0], filters[0])
        self.conv1_0 = conv_block_nested(filters[0], filters[1], filters[1])
        self.conv2_0 = conv_block_nested(filters[1], filters[2], filters[2])
        self.conv3_0 = conv_block_nested(filters[2], filters[3], filters[3])
        self.conv4_0 = conv_block_nested(filters[3], filters[4], filters[4])

        self.conv0_1 = conv_block_nested(filters[0] + filters[1], filters[0], filters[0])
        self.conv1_1 = conv_block_nested(filters[1] + filters[2], filters[1], filters[1])
        self.conv2_1 = conv_block_nested(filters[2] + filters[3], filters[2], filters[2])
        self.conv3_1 = conv_block_nested(filters[3] + filters[4], filters[3], filters[3])

        self.conv0_2 = conv_block_nested(filters[0]*2 + filters[1], filters[0], filters[0])
        self.conv1_2 = conv_block_nested(filters[1]*2 + filters[2], filters[1], filters[1])
        self.conv2_2 = conv_block_nested(filters[2]*2 + filters[3], filters[2], filters[2])

        self.conv0_3 = conv_block_nested(filters[0]*3 + filters[1], filters[0], filters[0])
        self.conv1_3 = conv_block_nested(filters[1]*3 + filters[2], filters[1], filters[1])

        self.conv0_4 = conv_block_nested(filters[0]*4 + filters[1], filters[0], filters[0])

        self.final = nn.Conv2d(filters[0], out_ch, kernel_size=1)

    def forward(self, x):

        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.Up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.Up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.Up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.Up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.Up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.Up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.Up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.Up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.Up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.Up(x1_3)], 1))

        output = self.final(x0_4)
        return output

## Unet++ ghost

In [133]:
class GhostModule(nn.Module):
    def __init__(self, inp, oup, kernel_size=1, ratio=2, dw_size=3, stride=1, relu=True, dilated=None):
        super(GhostModule, self).__init__()
        self.oup = oup
        init_channels = math.ceil(oup / ratio)
        new_channels = init_channels*(ratio-1)
        self.primary_conv = nn.Sequential(
            nn.Conv2d(inp, init_channels, kernel_size, stride, kernel_size//2, bias=False),
            nn.BatchNorm2d(init_channels),
            nn.ReLU(inplace=True) if relu else nn.Sequential(),
        )

        self.cheap_operation = nn.Sequential(
            nn.Conv2d(init_channels, new_channels, dw_size, 1, dw_size//2, groups=init_channels, bias=False),
            nn.BatchNorm2d(new_channels),
            nn.ReLU(inplace=True) if relu else nn.Sequential(),
        )

    def forward(self, x):
        x1 = self.primary_conv(x)
        x2 = self.cheap_operation(x1)
        out = torch.cat([x1,x2], dim=1)
        return out[:,:self.oup,:,:]
    
class conv_block_nested(nn.Module):

    def __init__(self, in_ch, mid_ch, out_ch):
        super(conv_block_nested, self).__init__()
        self.activation = nn.ReLU(inplace=True)
        self.conv1 = GhostModule(in_ch, mid_ch, kernel_size=3, relu=True)
#         self.conv3x1_1 = nn.Conv2d(in_ch, in_ch, kernel_size=(3,1), padding=(1,0), bias=True)
#         self.conv1x3_1 = nn.Conv2d(in_ch, mid_ch, kernel_size=(1,3), padding=(0,1), bias=True)
        
#         self.conv1 = nn.Conv2d(in_ch, mid_ch, kernel_size=3, padding=1, bias=True)
#         self.bn1 = nn.BatchNorm2d(mid_ch)
        
        self.conv2 = GhostModule(mid_ch, out_ch, kernel_size=3, relu=True)
#         self.conv3x1_2 = nn.Conv2d(mid_ch, mid_ch, kernel_size=(3,1), padding=(1,0), bias=True)
#         self.conv1x3_2 = nn.Conv2d(mid_ch, out_ch, kernel_size=(1,3), padding=(0,1), bias=True)
#         self.conv2 = nn.Conv2d(mid_ch, out_ch, kernel_size=3, padding=1, bias=True)
#         self.bn2 = nn.BatchNorm2d(out_ch)

    def forward(self, x):
        x = self.conv1(x)
#         x = self.conv3x1_1(x)
#         x = self.conv1x3_1(x)
        
#         x = self.bn1(x)
#         x = self.activation(x)

        x = self.conv2(x)
#         x = self.conv3x1_2(x)
#         x = self.conv1x3_2(x)
#         x = self.bn2(x)
#         output = self.activation(x)

        return x

class Nested_UNet(nn.Module):

    def __init__(self, in_ch=3, out_ch=1):
        super(Nested_UNet, self).__init__()

        n1 = 32
        filters = [n1, n1 * 2, n1 * 4, n1 * 8, n1 * 16]

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv0_0 = conv_block_nested(in_ch, filters[0], filters[0])
        self.conv1_0 = conv_block_nested(filters[0], filters[1], filters[1])
        self.conv2_0 = conv_block_nested(filters[1], filters[2], filters[2])
        self.conv3_0 = conv_block_nested(filters[2], filters[3], filters[3])
        self.conv4_0 = conv_block_nested(filters[3], filters[4], filters[4])

        self.conv0_1 = conv_block_nested(filters[0] + filters[1], filters[0], filters[0])
        self.conv1_1 = conv_block_nested(filters[1] + filters[2], filters[1], filters[1])
        self.conv2_1 = conv_block_nested(filters[2] + filters[3], filters[2], filters[2])
        self.conv3_1 = conv_block_nested(filters[3] + filters[4], filters[3], filters[3])

        self.conv0_2 = conv_block_nested(filters[0]*2 + filters[1], filters[0], filters[0])
        self.conv1_2 = conv_block_nested(filters[1]*2 + filters[2], filters[1], filters[1])
        self.conv2_2 = conv_block_nested(filters[2]*2 + filters[3], filters[2], filters[2])

        self.conv0_3 = conv_block_nested(filters[0]*3 + filters[1], filters[0], filters[0])
        self.conv1_3 = conv_block_nested(filters[1]*3 + filters[2], filters[1], filters[1])

        self.conv0_4 = conv_block_nested(filters[0]*4 + filters[1], filters[0], filters[0])

        self.final = nn.Conv2d(filters[0], out_ch, kernel_size=1)

    def forward(self, x):

        x0_0 = self.conv0_0(x)
        x1_0 = self.conv1_0(self.pool(x0_0))
        x0_1 = self.conv0_1(torch.cat([x0_0, self.Up(x1_0)], 1))

        x2_0 = self.conv2_0(self.pool(x1_0))
        x1_1 = self.conv1_1(torch.cat([x1_0, self.Up(x2_0)], 1))
        x0_2 = self.conv0_2(torch.cat([x0_0, x0_1, self.Up(x1_1)], 1))

        x3_0 = self.conv3_0(self.pool(x2_0))
        x2_1 = self.conv2_1(torch.cat([x2_0, self.Up(x3_0)], 1))
        x1_2 = self.conv1_2(torch.cat([x1_0, x1_1, self.Up(x2_1)], 1))
        x0_3 = self.conv0_3(torch.cat([x0_0, x0_1, x0_2, self.Up(x1_2)], 1))

        x4_0 = self.conv4_0(self.pool(x3_0))
        x3_1 = self.conv3_1(torch.cat([x3_0, self.Up(x4_0)], 1))
        x2_2 = self.conv2_2(torch.cat([x2_0, x2_1, self.Up(x3_1)], 1))
        x1_3 = self.conv1_3(torch.cat([x1_0, x1_1, x1_2, self.Up(x2_2)], 1))
        x0_4 = self.conv0_4(torch.cat([x0_0, x0_1, x0_2, x0_3, self.Up(x1_3)], 1))

        output = self.final(x0_4)
        return output

## Deeplab v3+ mobilenet

In [23]:
def conv_bn(inp, oup, stride, BatchNorm):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        BatchNorm(oup),
        nn.ReLU6(inplace=True)
    )


def fixed_padding(inputs, kernel_size, dilation):
    kernel_size_effective = kernel_size + (kernel_size - 1) * (dilation - 1)
    pad_total = kernel_size_effective - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg
    padded_inputs = F.pad(inputs, (pad_beg, pad_end, pad_beg, pad_end))
    return padded_inputs


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, dilation, expand_ratio, BatchNorm):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.use_res_connect = self.stride == 1 and inp == oup
        self.kernel_size = 3
        self.dilation = dilation

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 0, dilation, groups=hidden_dim, bias=False),
                BatchNorm(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, 1, 1, bias=False),
                BatchNorm(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, 1, bias=False),
                BatchNorm(hidden_dim),
                nn.ReLU6(inplace=True),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 0, dilation, groups=hidden_dim, bias=False),
                BatchNorm(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, 1, bias=False),
                BatchNorm(oup),
            )

    def forward(self, x):
        x_pad = fixed_padding(x, self.kernel_size, dilation=self.dilation)
        if self.use_res_connect:
            x = x + self.conv(x_pad)
        else:
            x = self.conv(x_pad)
        return x


class MobileNetV2(nn.Module):
    def __init__(self, output_stride=16, BatchNorm=None, width_mult=1., pretrained=False):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        current_stride = 1
        rate = 1
        interverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # building first layer
        input_channel = int(input_channel * width_mult)
        self.features = [conv_bn(3, input_channel, 2, BatchNorm)]
        current_stride *= 2
        # building inverted residual blocks
        for t, c, n, s in interverted_residual_setting:
            if current_stride == output_stride:
                stride = 1
                dilation = rate
                rate *= s
            else:
                stride = s
                dilation = 1
                current_stride *= s
            output_channel = int(c * width_mult)
            for i in range(n):
                if i == 0:
                    self.features.append(block(input_channel, output_channel, stride, dilation, t, BatchNorm))
                else:
                    self.features.append(block(input_channel, output_channel, 1, dilation, t, BatchNorm))
                input_channel = output_channel
        self.features = nn.Sequential(*self.features)
        self._initialize_weights()

        if pretrained:
            self._load_pretrained_model()
        print(len(self.features))
        self.low_level_features = self.features[0:4]
        self.high_level_features = self.features[4:]
        

    def forward(self, x):
        low_level_feat = self.low_level_features(x)
#         print('low_level_feat shape', low_level_feat.shape)
        x = self.high_level_features(low_level_feat)
#         print('x1 shape', x.shape)

        return x, low_level_feat

    def _load_pretrained_model(self):
        pretrain_dict = model_zoo.load_url('http://jeff95.me/models/mobilenet_v2-6a65762b.pth')
        model_dict = {}
        state_dict = self.state_dict()
        for k, v in pretrain_dict.items():
            if k in state_dict:
                model_dict[k] = v
        state_dict.update(model_dict)
        self.load_state_dict(state_dict)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                # m.weight.data.normal_(0, math.sqrt(2. / n))
                torch.nn.init.kaiming_normal_(m.weight)
            # elif isinstance(m, SynchronizedBatchNorm2d):
            #     m.weight.data.fill_(1)
            #     m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()


class ASPP_module(nn.Module):
    def __init__(self, inplanes, planes, rate):
        super(ASPP_module, self).__init__()
        if rate == 1:
            kernel_size = 1
            padding = 0
        else:
            kernel_size = 3
            padding = rate
        self.atrous_convolution = nn.Conv2d(inplanes, planes, kernel_size=kernel_size,
                                            stride=1, padding=padding, dilation=rate, bias=False)
        self.bn = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU()

        self.__init_weight()

    def forward(self, x):
        x = self.atrous_convolution(x)
        x = self.bn(x)

        return self.relu(x)

    def __init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                # m.weight.data.normal_(0, math.sqrt(2. / n))
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                
                
class DeepLabv3_plus_mobilenet(nn.Module):
    def __init__(self, nInputChannels=3, n_classes=1, os=16, pretrained=False, _print=True):
        if _print:
            print("Constructing DeepLabv3+ model...")
            print("Number of classes: {}".format(n_classes))
            print("Output stride: {}".format(os))
            print("Number of Input Channels: {}".format(nInputChannels))
        super(DeepLabv3_plus_mobilenet, self).__init__()

        # Atrous Conv
        self.efficient_features = MobileNetV2(output_stride=16, BatchNorm=nn.BatchNorm2d)

        # ASPP
        if os == 16:
            rates = [1, 6, 12, 18]
        elif os == 8:
            rates = [1, 12, 24, 36]
        else:
            raise NotImplementedError

        self.aspp1 = ASPP_module(320, 256, rate=rates[0])
        self.aspp2 = ASPP_module(320, 256, rate=rates[1])
        self.aspp3 = ASPP_module(320, 256, rate=rates[2])
        self.aspp4 = ASPP_module(320, 256, rate=rates[3])

        self.relu = nn.ReLU()

        self.global_avg_pool = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                                             nn.Conv2d(320, 256, 1, stride=1, bias=False),
                                             nn.BatchNorm2d(256),
                                             nn.ReLU())

        self.conv1 = nn.Conv2d(1280, 256, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(256)

        # adopt [1x1, 48] for channel reduction.
        self.conv2 = nn.Conv2d(24, 48, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(48)

        self.last_conv = nn.Sequential(nn.Conv2d(304, 256, kernel_size=3, stride=1, padding=1, bias=False),
                                       nn.BatchNorm2d(256),
                                       nn.ReLU(),
                                       nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False),
                                       nn.BatchNorm2d(256),
                                       nn.ReLU(),
                                       nn.Conv2d(256, n_classes, kernel_size=1, stride=1))

    def forward(self, input):
        x, low_level_features = self.efficient_features(input)
        x1 = self.aspp1(x)
        x2 = self.aspp2(x)
        x3 = self.aspp3(x)
        x4 = self.aspp4(x)
        x5 = self.global_avg_pool(x)
        x5 = F.upsample(x5, size=x4.size()[2:], mode='bilinear', align_corners=True)

        x = torch.cat((x1, x2, x3, x4, x5), dim=1)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = F.upsample(x, size=(int(math.ceil(input.size()[-2]/4)),
                                int(math.ceil(input.size()[-1]/4))), mode='bilinear', align_corners=True)

        low_level_features = self.conv2(low_level_features)
        low_level_features = self.bn2(low_level_features)
        low_level_features = self.relu(low_level_features)


        x = torch.cat((x, low_level_features), dim=1)
        x = self.last_conv(x)
        x = F.upsample(x, size=input.size()[2:], mode='bilinear', align_corners=True)

        return x

    def freeze_bn(self):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    def __init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                # torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()





## Deeplab V3+ mobilenet ghost

In [137]:
class GhostModule(nn.Module):
    def __init__(self, inp, oup, kernel_size=1, ratio=2, dw_size=3, stride=1, relu=True, is_relu6=False):
        super(GhostModule, self).__init__()
        self.oup = oup
        init_channels = math.ceil(oup / ratio)
        new_channels = init_channels*(ratio-1)
        if is_relu6:
            self.primary_conv = nn.Sequential(
                nn.Conv2d(inp, init_channels, kernel_size, stride, kernel_size//2, bias=False),
                nn.BatchNorm2d(init_channels),
                nn.ReLU6(inplace=True) if relu else nn.Sequential(),
            )

            self.cheap_operation = nn.Sequential(
                nn.Conv2d(init_channels, new_channels, dw_size, 1, dw_size//2, groups=init_channels, bias=False),
                nn.BatchNorm2d(new_channels),
                nn.ReLU6(inplace=True) if relu else nn.Sequential(),
            )
        else:
            self.primary_conv = nn.Sequential(
                nn.Conv2d(inp, init_channels, kernel_size, stride, kernel_size//2, bias=False),
                nn.BatchNorm2d(init_channels),
                nn.ReLU(inplace=True) if relu else nn.Sequential(),
            )

            self.cheap_operation = nn.Sequential(
                nn.Conv2d(init_channels, new_channels, dw_size, 1, dw_size//2, groups=init_channels, bias=False),
                nn.BatchNorm2d(new_channels),
                nn.ReLU(inplace=True) if relu else nn.Sequential(),
            )            
    def forward(self, x):
        x1 = self.primary_conv(x)
        x2 = self.cheap_operation(x1)
        out = torch.cat([x1,x2], dim=1)
        return out[:,:self.oup,:,:]
    
def conv_bn(inp, oup, stride, BatchNorm):
    return nn.Sequential(
        GhostModule(inp, oup, kernel_size=3, stride=stride, relu=True, is_relu6=True)
#         nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
#         BatchNorm(oup),
#         nn.ReLU6(inplace=True)
    )


def fixed_padding(inputs, kernel_size, dilation):
    kernel_size_effective = kernel_size + (kernel_size - 1) * (dilation - 1)
    pad_total = kernel_size_effective - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg
    padded_inputs = F.pad(inputs, (pad_beg, pad_end, pad_beg, pad_end))
    return padded_inputs


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, dilation, expand_ratio, BatchNorm):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.use_res_connect = self.stride == 1 and inp == oup
        self.kernel_size = 3
        self.dilation = dilation

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 0, dilation, groups=hidden_dim, bias=False),
                BatchNorm(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, 1, 1, bias=False),
                BatchNorm(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, 1, bias=False),
                BatchNorm(hidden_dim),
                nn.ReLU6(inplace=True),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 0, dilation, groups=hidden_dim, bias=False),
                BatchNorm(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, 1, bias=False),
                BatchNorm(oup),
            )

    def forward(self, x):
        x_pad = fixed_padding(x, self.kernel_size, dilation=self.dilation)
        if self.use_res_connect:
            x = x + self.conv(x_pad)
        else:
            x = self.conv(x_pad)
        return x


class MobileNetV2(nn.Module):
    def __init__(self, output_stride=16, BatchNorm=None, width_mult=1., pretrained=False):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        current_stride = 1
        rate = 1
        interverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # building first layer
        input_channel = int(input_channel * width_mult)
        self.features = [conv_bn(3, input_channel, 2, BatchNorm)]
        current_stride *= 2
        # building inverted residual blocks
        for t, c, n, s in interverted_residual_setting:
            if current_stride == output_stride:
                stride = 1
                dilation = rate
                rate *= s
            else:
                stride = s
                dilation = 1
                current_stride *= s
            output_channel = int(c * width_mult)
            for i in range(n):
                if i == 0:
                    self.features.append(block(input_channel, output_channel, stride, dilation, t, BatchNorm))
                else:
                    self.features.append(block(input_channel, output_channel, 1, dilation, t, BatchNorm))
                input_channel = output_channel
        self.features = nn.Sequential(*self.features)
        self._initialize_weights()

        if pretrained:
            self._load_pretrained_model()

        self.low_level_features = self.features[0:4]
        self.high_level_features = self.features[4:]

    def forward(self, x):
        low_level_feat = self.low_level_features(x)
        x = self.high_level_features(low_level_feat)
        return x, low_level_feat

    def _load_pretrained_model(self):
        pretrain_dict = model_zoo.load_url('http://jeff95.me/models/mobilenet_v2-6a65762b.pth')
        model_dict = {}
        state_dict = self.state_dict()
        for k, v in pretrain_dict.items():
            if k in state_dict:
                model_dict[k] = v
        state_dict.update(model_dict)
        self.load_state_dict(state_dict)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                # m.weight.data.normal_(0, math.sqrt(2. / n))
                torch.nn.init.kaiming_normal_(m.weight)
            # elif isinstance(m, SynchronizedBatchNorm2d):
            #     m.weight.data.fill_(1)
            #     m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

class ASPP_module(nn.Module):
    def __init__(self, inplanes, planes, rate):
        super(ASPP_module, self).__init__()
        if rate == 1:
            kernel_size = 1
            padding = 0
        else:
            kernel_size = 3
            padding = rate
        self.atrous_convolution = nn.Conv2d(inplanes, planes, kernel_size=kernel_size,
                                            stride=1, padding=padding, dilation=rate, bias=False)
        self.bn = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU()

        self.__init_weight()

    def forward(self, x):
        x = self.atrous_convolution(x)
        x = self.bn(x)

        return self.relu(x)

    def __init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                # m.weight.data.normal_(0, math.sqrt(2. / n))
                torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
                
class DeepLabv3_plus_mobilenet(nn.Module):
    def __init__(self, nInputChannels=3, n_classes=1, os=16, pretrained=False, _print=True):
        if _print:
            print("Constructing DeepLabv3+ model...")
            print("Number of classes: {}".format(n_classes))
            print("Output stride: {}".format(os))
            print("Number of Input Channels: {}".format(nInputChannels))
        super(DeepLabv3_plus_mobilenet, self).__init__()

        # Atrous Conv
        self.efficient_features = MobileNetV2(output_stride=16, BatchNorm=nn.BatchNorm2d)

        # ASPP
        if os == 16:
            rates = [1, 6, 12, 18]
        elif os == 8:
            rates = [1, 12, 24, 36]
        else:
            raise NotImplementedError

        self.aspp1 = ASPP_module(320, 256, rate=rates[0])
        self.aspp2 = ASPP_module(320, 256, rate=rates[1])
        self.aspp3 = ASPP_module(320, 256, rate=rates[2])
        self.aspp4 = ASPP_module(320, 256, rate=rates[3])

        self.relu = nn.ReLU()

        self.global_avg_pool = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                                             GhostModule(320, 256, kernel_size=1, stride=1, relu=True)
#                                              nn.Conv2d(320, 256, 1, stride=1, bias=False),
#                                              nn.BatchNorm2d(256),
#                                              nn.ReLU()
                                            )

        self.conv1 = nn.Conv2d(1280, 256, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(256)

        # adopt [1x1, 48] for channel reduction.
        self.conv2 = nn.Conv2d(24, 48, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(48)

        self.last_conv = nn.Sequential(GhostModule(304, 256, kernel_size=3, stride=1, relu=True),
                                       GhostModule(256, 256, kernel_size=3, stride=1, relu=True),
#             nn.Conv2d(304, 256, kernel_size=3, stride=1, padding=1, bias=False),
#                                        nn.BatchNorm2d(256),
#                                        nn.ReLU(),
#                                        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1, bias=False),
#                                        nn.BatchNorm2d(256),
#                                        nn.ReLU(),
                                       nn.Conv2d(256, n_classes, kernel_size=1, stride=1))

    def forward(self, input):
        x, low_level_features = self.efficient_features(input)
        x1 = self.aspp1(x)
        x2 = self.aspp2(x)
        x3 = self.aspp3(x)
        x4 = self.aspp4(x)
        x5 = self.global_avg_pool(x)
        x5 = F.upsample(x5, size=x4.size()[2:], mode='bilinear', align_corners=True)

        x = torch.cat((x1, x2, x3, x4, x5), dim=1)

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = F.upsample(x, size=(int(math.ceil(input.size()[-2]/4)),
                                int(math.ceil(input.size()[-1]/4))), mode='bilinear', align_corners=True)

        low_level_features = self.conv2(low_level_features)
        low_level_features = self.bn2(low_level_features)
        low_level_features = self.relu(low_level_features)


        x = torch.cat((x, low_level_features), dim=1)
        x = self.last_conv(x)
        x = F.upsample(x, size=input.size()[2:], mode='bilinear', align_corners=True)

        return x

    def freeze_bn(self):
        for m in self.modules():
            if isinstance(m, nn.BatchNorm2d):
                m.eval()

    def __init_weight(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                # torch.nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()





# Helper

In [4]:
def read_images(root_path): 
  """ 
    從目錄資料夾中讀出所有檔名 
    Args : directory path
    Return : list with file path
  """

  for root, dirs, files in os.walk(root_path):
    print("path：", root)
    print("directory：", dirs)
    print("file：", files)
    
  org_imgs = [(root + "/" + file_name) for file_name in tqdm(files)]
  org_imgs.sort()
  print("Read ",len(org_imgs), " files")
  return org_imgs

def make_cm2lbl():

  colormap = [[0,0,0], [0,0,128], [0,128,128], [128,0,0], [0,128,0]] # background, grider, net, lanyard, guardrail 

  cm2lbl = np.zeros(256**3) # for each pixel 0~255 , channel = 3(RGB)
  for i,cm in enumerate(colormap):
      cm2lbl[(cm[0]*256+cm[1])*256+cm[2]] = i
      # print(i)
      # print((cm[0]*256+cm[1])*256+cm[2]
  return cm2lbl


def image2label(img, cm2lbl):
  data = np.array(img, dtype='int32')
  idx = (data[:, :, 0] * 256 + data[:, :, 1]) * 256 + data[:, :, 2]
  # print(np.unique(idx))
  # print(cm2lbl[idx])
  result = np.array(cm2lbl[idx], dtype='int64')  
  return result[:,:,None]

# helper function for data visualization
def visualize(**images):
  """PLot images in one row."""
  n = len(images)
  plt.figure(figsize=(16, 5))
  for i, (name, image) in enumerate(images.items()):
    plt.subplot(1, n, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.title(' '.join(name.split('_')).title())
    plt.imshow(image)
  plt.show()

def print_network(net):
  num_params = 0
  for param in net.parameters():
    num_params += param.numel()
  print(net)
  print('Total number of parameters: %d' % num_params)

In [5]:
# Taken from here
# https://github.com/wanglouis49/pytorch-weights_pruning/blob/master/pruning/layers.py

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


class LinearMasked(nn.Linear):
    def __init__(self, in_features, out_features, bias=True):
        super(LinearMasked, self).__init__(in_features, out_features, bias)
        self.mask_flag = False

    def set_mask(self, mask):
        self.mask = Variable(mask, requires_grad=False, volatile=False)
        self.weight.data = self.weight.data * self.mask.data
        self.mask_flag = True

    def get_mask(self):
        print(self.mask_flag)
        return self.mask

    def forward(self, x):
        if self.mask_flag:
            weight = self.weight * self.mask
            return F.linear(x, weight, self.bias)
        else:
            return F.linear(x, self.weight, self.bias)


class Conv2dMasked(nn.Conv2d):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride=1,
        padding=0,
        dilation=1,
        groups=1,
        bias=True,
    ):
        super(Conv2dMasked, self).__init__(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            dilation,
            groups,
            bias,
        )
        self.mask_flag = False

    def set_mask(self, mask):
        self.mask = Variable(mask, requires_grad=False, volatile=False)
        self.weight.data = self.weight.data * self.mask.data
        self.mask_flag = True

    def get_mask(self):
        print(self.mask_flag)
        return self.mask

    def forward(self, x):
        if self.mask_flag:
            weight = self.weight * self.mask
            return F.conv2d(
                x,
                weight,
                self.bias,
                self.stride,
                self.padding,
                self.dilation,
                self.groups,
            )
        else:
            return F.conv2d(
                x,
                self.weight,
                self.bias,
                self.stride,
                self.padding,
                self.dilation,
                self.groups,
            )

In [6]:
def dice_loss(pred, target, smooth = 1.):
    pred = pred.contiguous()
    target = target.contiguous()    

    intersection = (pred * target).sum(dim=2).sum(dim=2)
    
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=2).sum(dim=2) + target.sum(dim=2).sum(dim=2) + smooth)))
    
    return loss.mean()

def calc_loss(pred, target, metrics, bce_weight=0.5):
    bce = F.binary_cross_entropy_with_logits(pred, target)
        
    pred = F.sigmoid(pred)
    dice = dice_loss(pred, target)
    
    loss = bce * bce_weight + dice * (1 - bce_weight)
    
    metrics['bce'] += bce.data.cpu().numpy() * target.size(0)
    metrics['dice'] += dice.data.cpu().numpy() * target.size(0)
    metrics['loss'] += loss.data.cpu().numpy() * target.size(0)
    
    return loss

def print_metrics(metrics, epoch_samples, phase):    
    outputs = []
    for k in metrics.keys():
        outputs.append("{}: {:4f}".format(k, metrics[k] / epoch_samples))
        
    print("{}: {}".format(phase, ", ".join(outputs))) 

In [7]:
# F.iou(y_pr, y_gt, eps=1e-7, threshold=0.5, ignore_channels=None)
def _take_channels(*xs, ignore_channels=None):
    if ignore_channels is None:
        return xs
    else:
        channels = [channel for channel in range(xs[0].shape[1]) if channel not in ignore_channels]
        xs = [torch.index_select(x, dim=1, index=torch.tensor(channels).to(x.device)) for x in xs]
        return xs


def _threshold(x, threshold=None):
    if threshold is not None:
        return (x > threshold)
    else:
        return x
    
def iou(pr, gt, eps=1e-7, threshold=None, ignore_channels=None):
    """Calculate Intersection over Union between ground truth and prediction
    Args:
        pr (torch.Tensor): predicted tensor
        gt (torch.Tensor):  ground truth tensor
        eps (float): epsilon to avoid zero division
        threshold: threshold for outputs binarization
    Returns:
        float: IoU (Jaccard) score
    """

    pr = _threshold(pr, threshold=threshold)
    pr, gt = _take_channels(pr, gt, ignore_channels=ignore_channels)

    intersection = torch.sum(gt * pr)
    union = torch.sum(gt) + torch.sum(pr) - intersection + eps
    return (intersection + eps) / union


In [8]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.9),
    transforms.RandomAffine(degrees=0, translate=(0.005, 0.0025)),
])
transform_toTensor = transforms.Compose([
    transforms.ToTensor()
])

In [9]:
def ratioFunction(num1, num2):
    num1 = int(num1) # Now we are good
    num2 = int(num2) # Good, good
    if num1 > num2:
        ratio12 = num2/num1
    else:
        ratio12 = num1/num2
#     print('The ratio of', num1, 'and', num2,'is', ratio12 + '.')
    return ratio12
ratioFunction(288, 216)

0.75

In [10]:
def test_lanyard_re(best_model, classes, test_features_folder, test_labels_folder):
  iou_scores = []
  iou_scores_resized = []
#   best_model = torch.load(weight_path + '/best_model.pth')
  test_input_imgs = read_images(test_features_folder)
  test_output_imgs = read_images(test_labels_folder)

  cm2lbl = make_cm2lbl()

  test_dataset = IS_Dataset_test(
      test_input_imgs, 
      test_output_imgs, 
#       transform=transform,
      transform_toTensor=transform_toTensor,
      classes=classes,
      cm2lbl=cm2lbl
   )
  test_dataset_vis = IS_Dataset_test(test_input_imgs, test_output_imgs, classes=classes, cm2lbl=cm2lbl)

  test_resized_dataset = IS_Dataset(
      test_input_imgs, 
      test_output_imgs, 
#       transform=transform,
      transform_toTensor=transform_toTensor,
      classes=classes,
      cm2lbl=cm2lbl,resize_size=(512,512)
  )
  
  
  for i in range(len(test_dataset)):
    image_vis = test_dataset_vis[i][0].astype('uint8')

    image, gt_mask = test_dataset[i]
    image_resized, gt_mask_resized = test_resized_dataset[i]
    
    gt_mask = gt_mask.squeeze()
#     gt_mask_tensor = torch.from_numpy(gt_mask)
    gt_mask_tensor = gt_mask
    
    gt_mask_resized = gt_mask_resized.squeeze()
    
#     x_tensor = torch.from_numpy(image).to('cuda').unsqueeze(0)
    x_tensor = image.to('cuda').unsqueeze(0)
    pr_mask = best_model(x_tensor).data
    pr_mask = torch.sigmoid(pr_mask)
    pr_mask = (pr_mask.squeeze().cpu().numpy().round())
    pr_mask_tensor = torch.from_numpy(pr_mask)
    
#     x_tensor_resized = torch.from_numpy(image_resized).to('cuda').unsqueeze(0)
    x_tensor_resized = image_resized.to('cuda').unsqueeze(0)
    pr_mask_resized = best_model(x_tensor_resized).data
    pr_mask_resized = torch.sigmoid(pr_mask_resized)
    pr_mask_resized = (pr_mask_resized.squeeze().cpu().numpy().round())
    pr_mask_resized_return = cv2.resize(pr_mask_resized, (gt_mask.shape[1], gt_mask.shape[0]), interpolation=cv2.INTER_NEAREST)
    pr_mask_resized_return_tensor = torch.from_numpy(pr_mask_resized_return)
    
    iou_score = iou(pr_mask_tensor, gt_mask_tensor, eps=1e-7, threshold=0.5, ignore_channels=None)
    iou_score_resized = iou(pr_mask_resized_return_tensor, gt_mask_tensor, eps=1e-7, threshold=0.5, ignore_channels=None)
    iou_scores.append(iou_score)
    iou_scores_resized.append(iou_score_resized)
    

  return np.mean(iou_scores), np.mean(iou_scores_resized)

In [11]:
test_features_folder = '/home/user/Desktop/lanyard_segmentation/guardrail/test_data/features'
test_labels_folder = '/home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels'
orig_features_folder = '/home/user/Desktop/lanyard_segmentation/guardrail/orig_features'
orig_labels_folder = '/home/user/Desktop/lanyard_segmentation/guardrail/orig_labels'
train_features_folder = "/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped"
train_labels_folder = "/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped"
viz_classes = ['guardrail']
classes = ['guardrail']

In [12]:
def get_loaders():
    train_features_folder = "/home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped"
    train_labels_folder = "/home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped"
    
    input_imgs = read_images(train_features_folder)
    output_imgs = read_images(train_labels_folder)
    classes = ["guardrail"]
    cm2lbl = make_cm2lbl()
    
    input_train, input_vali, output_train, output_vali = train_test_split(input_imgs, output_imgs, test_size=0.2, random_state=42)

    train_dataset = IS_Dataset(
      input_train, 
      output_train, 
      transform=transform,
      transform_toTensor=transform_toTensor,
      classes=classes,
      cm2lbl=cm2lbl
      )
    validation_dataset = IS_Dataset(
      input_vali, 
      output_vali,
      classes=classes,
      transform=transform, 
      transform_toTensor=transform_toTensor,
      cm2lbl=cm2lbl
      )

    train_dataloader = DataLoader(train_dataset, 2, shuffle=True, num_workers=2, drop_last=True)
    validation_dataloader = DataLoader(validation_dataset, 1, shuffle=False, num_workers=2)
    
    return train_dataloader, validation_dataloader

In [13]:
def nonzero(tensor):
    """Returns absolute number of values different from 0
    Arguments:
        tensor {numpy.ndarray} -- Array to compute over
    Returns:
        int -- Number of nonzero elements
    """
    return np.sum(tensor != 0.0)


# https://pytorch.org/docs/stable/tensor_attributes.html
dtype2bits = {
    torch.float32: 32,
    torch.float: 32,
    torch.float64: 64,
    torch.double: 64,
    torch.float16: 16,
    torch.half: 16,
    torch.uint8: 8,
    torch.int8: 8,
    torch.int16: 16,
    torch.short: 16,
    torch.int32: 32,
    torch.int: 32,
    torch.int64: 64,
    torch.long: 64,
    torch.bool: 1,
}


def model_size(model, as_bits=False):
    """Returns absolute and nonzero model size
    Arguments:
        model {torch.nn.Module} -- Network to compute model size over
    Keyword Arguments:
        as_bits {bool} -- Whether to account for the size of dtype
    Returns:
        int -- Total number of weight & bias params
        int -- Out total_params exactly how many are nonzero
    """

    total_params = 0
    nonzero_params = 0
    for tensor in model.parameters():
        t = np.prod(tensor.shape)
        nz = nonzero(tensor.detach().cpu().numpy())
        if as_bits:
            bits = dtype2bits[tensor.dtype]
            t *= bits
            nz *= bits
        total_params += t
        nonzero_params += nz
    return int(total_params), int(nonzero_params)

In [14]:
def hook_applyfn(hook, model, forward=False, backward=False):
    """
    [description]
    Arguments:
        hook {[type]} -- [description]
        model {[type]} -- [description]
    Keyword Arguments:
        forward {bool} -- [description] (default: {False})
        backward {bool} -- [description] (default: {False})
    Returns:
        [type] -- [description]
    """
    assert forward ^ backward, "Either forward or backward must be True"
    hooks = []

    def register_hook(module):
        if (
            not isinstance(module, nn.Sequential)
            and not isinstance(module, nn.ModuleList)
            and not isinstance(module, nn.ModuleDict)
            and not (module == model)
        ):
            if forward:
                hooks.append(module.register_forward_hook(hook))
            if backward:
                hooks.append(module.register_backward_hook(hook))

    return register_hook, hooks


def get_params(model, recurse=False):
    """Returns dictionary of paramters
    Arguments:
        model {torch.nn.Module} -- Network to extract the parameters from
    Keyword Arguments:
        recurse {bool} -- Whether to recurse through children modules
    Returns:
        Dict(str:numpy.ndarray) -- Dictionary of named parameters their
                                   associated parameter arrays
    """
    params = {
        k: v.detach().cpu().numpy().copy()
        for k, v in model.named_parameters(recurse=recurse)
    }
    return params


def get_activations(model, input):

    activations = OrderedDict()

    def store_activations(module, input, output):
        if isinstance(module, nn.ReLU):
            # TODO ResNet18 implementation reuses a
            # single ReLU layer?
            return
#         assert module not in activations, f"{module} already in activations"
        # TODO [0] means first input, not all models have a single input
#         print('module:', module)
        if isinstance(module, MobileNetV2):
#             print()
            activations[module] = (
                input[0].detach().cpu().numpy().copy(),
                output[0].detach().cpu().numpy().copy(),
            )
        else:
            activations[module] = (
                input[0].detach().cpu().numpy().copy(),
                output.detach().cpu().numpy().copy(),
            )
    fn, hooks = hook_applyfn(store_activations, model, forward=True)
    model.apply(fn)
    with torch.no_grad():
        model(input)

    for h in hooks:
        h.remove()

    return activations


def get_gradients(model, inputs, outputs):
    # TODO implement using model.register_backward_hook()
    # So it is harder than it seems, the grad_input contains also the gradients
    # with respect to the weights and so far order seems to be
    # (bias, input, weight) which is confusing.
    # Moreover, a lot of the time the output activation we are
    # looking for is the one after the ReLU and F.ReLU (or any functional call)
    # will not be called by the forward or backward hook
    # Discussion here
    # https://discuss.pytorch.org/t/how-to-register-hook-function-for-functional-form/25775
    # Best way seems to be monkey patching F.ReLU & other functional ops
    # That'll also help figuring out how to compute a module graph
    pass


def get_param_gradients(model, inputs, outputs, loss_func=None, by_module=True):

    gradients = OrderedDict()

    if loss_func is None:
        loss_func = nn.CrossEntropyLoss()

    training = model.training
    model.train()
    pred = model(inputs)
    loss = loss_func(pred, outputs)
    loss.backward()

    if by_module:
        gradients = defaultdict(OrderedDict)
        for module in model.modules():
            assert module not in gradients
            for name, param in module.named_parameters(recurse=False):
                if param.requires_grad and param.grad is not None:
                    gradients[module][name] = param.grad.detach().cpu().numpy().copy()

    else:
        gradients = OrderedDict()
        for name, param in model.named_parameters():
            assert name not in gradients
            if param.requires_grad and param.grad is not None:
                gradients[name] = param.grad.detach().cpu().numpy().copy()

    model.zero_grad()
    model.train(training)

    return gradients


def fraction_to_keep(compression, model, prunable_modules):
    """Return fraction of params to keep to achieve desired compression ratio
    Compression = total / ( fraction * prunable + (total-prunable))
    Using algrebra fraction is equal to
    fraction = total/prunable * (1/compression - 1) + 1
    Arguments:
        compression {float} -- Desired overall compression
        model {torch.nn.Module} -- Full model for which to compute the fraction
        prunable_modules {List(torch.nn.Module)} --
        Modules that can be pruned in the model.
    Returns:
        {float} -- Fraction of prunable parameters to keep
        to achieve desired compression
    """
    from ..metrics import model_size

    total_size, _ = model_size(model)
    prunable_size = sum([model_size(m)[0] for m in prunable_modules])
    nonprunable_size = total_size - prunable_size
    fraction = 1 / prunable_size * (total_size / compression - nonprunable_size)
    assert 0 < fraction <= 1, (
        f"Cannot compress to {1/compression} model\
            with {nonprunable_size/total_size}"
        + "fraction of unprunable parameters"
    )
    return fraction

In [15]:
def dense_flops(in_neurons, out_neurons):
    """Compute the number of multiply-adds used by a Dense (Linear) layer"""
    return in_neurons * out_neurons


def conv2d_flops(
    in_channels,
    out_channels,
    input_shape,
    kernel_shape,
    padding="same",
    strides=1,
    dilation=1,
):
    """Compute the number of multiply-adds used by a Conv2D layer
    Args:
        in_channels (int): The number of channels in the layer's input
        out_channels (int): The number of channels in the layer's output
        input_shape (int, int): The spatial shape of the rank-3 input tensor
        kernel_shape (int, int): The spatial shape of the rank-4 kernel
        padding ({'same', 'valid'}): The padding used by the convolution
        strides (int) or (int, int): The spatial stride of the convolution;
            two numbers may be specified if it's different for the x and y axes
        dilation (int): Must be 1 for now.
    Returns:
        int: The number of multiply-adds a direct convolution would require
        (i.e., no FFT, no Winograd, etc)
    >>> c_in, c_out = 10, 10
    >>> in_shape = (4, 5)
    >>> filt_shape = (3, 2)
    >>> # valid padding
    >>> ret = conv2d_flops(c_in, c_out, in_shape, filt_shape, padding='valid')
    >>> ret == int(c_in * c_out * np.prod(filt_shape) * (2 * 4))
    True
    >>> # same padding, no stride
    >>> ret = conv2d_flops(c_in, c_out, in_shape, filt_shape, padding='same')
    >>> ret == int(c_in * c_out * np.prod(filt_shape) * np.prod(in_shape))
    True
    >>> # valid padding, stride > 1
    >>> ret = conv2d_flops(c_in, c_out, in_shape, filt_shape, \
                       padding='valid', strides=(1, 2))
    >>> ret == int(c_in * c_out * np.prod(filt_shape) * (2 * 2))
    True
    >>> # same padding, stride > 1
    >>> ret = conv2d_flops(c_in, c_out, in_shape, filt_shape, \
                           padding='same', strides=2)
    >>> ret == int(c_in * c_out * np.prod(filt_shape) * (2 * 3))
    True
    """
    # validate + sanitize input
    assert in_channels > 0
    assert out_channels > 0
    assert len(input_shape) == 2
    assert len(kernel_shape) == 2
    padding = padding.lower()
    assert padding in (
        "same",
        "valid",
        "zeros",
    ), "Padding must be one of same|valid|zeros"
    try:
        strides = tuple(strides)
    except TypeError:
        # if one number provided, make it a 2-tuple
        strides = (strides, strides)
#     assert dilation == 1 or all(
#         d == 1 for d in dilation
#     ), "Dilation > 1 is not supported"

    # compute output spatial shape
    # based on TF computations https://stackoverflow.com/a/37674568
    if padding in ["same", "zeros"]:
        out_nrows = np.ceil(float(input_shape[0]) / strides[0])
        out_ncols = np.ceil(float(input_shape[1]) / strides[1])
    else:  # padding == 'valid'
        out_nrows = np.ceil((input_shape[0] - kernel_shape[0] + 1) / strides[0])  # noqa
        out_ncols = np.ceil((input_shape[1] - kernel_shape[1] + 1) / strides[1])  # noqa
    output_shape = (int(out_nrows), int(out_ncols))

    # work to compute one output spatial position
    nflops = in_channels * out_channels * int(np.prod(kernel_shape))

    # total work = work per output position * number of output positions
    return nflops * int(np.prod(output_shape))

In [16]:
class LinearMasked(nn.Linear):
    def __init__(self, in_features, out_features, bias=True):
        super(LinearMasked, self).__init__(in_features, out_features, bias)
        self.mask_flag = False

    def set_mask(self, mask):
        self.mask = Variable(mask, requires_grad=False, volatile=False)
        self.weight.data = self.weight.data * self.mask.data
        self.mask_flag = True

    def get_mask(self):
        print(self.mask_flag)
        return self.mask

    def forward(self, x):
        if self.mask_flag:
            weight = self.weight * self.mask
            return F.linear(x, weight, self.bias)
        else:
            return F.linear(x, self.weight, self.bias)


class Conv2dMasked(nn.Conv2d):
    def __init__(
        self,
        in_channels,
        out_channels,
        kernel_size,
        stride=1,
        padding=0,
        dilation=1,
        groups=1,
        bias=True,
    ):
        super(Conv2dMasked, self).__init__(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            dilation,
            groups,
            bias,
        )
        self.mask_flag = False

    def set_mask(self, mask):
        self.mask = Variable(mask, requires_grad=False, volatile=False)
        self.weight.data = self.weight.data * self.mask.data
        self.mask_flag = True

    def get_mask(self):
        print(self.mask_flag)
        return self.mask

    def forward(self, x):
        if self.mask_flag:
            weight = self.weight * self.mask
            return F.conv2d(
                x,
                weight,
                self.bias,
                self.stride,
                self.padding,
                self.dilation,
                self.groups,
            )
        else:
            return F.conv2d(
                x,
                self.weight,
                self.bias,
                self.stride,
                self.padding,
                self.dilation,
                self.groups,
            )

In [17]:
def _conv2d_flops(module, activation):
    # Auxiliary func to use abstract flop computation

    # Drop batch & channels. Channels can be dropped since
    # unlike shape they have to match to in_channels
    input_shape = activation.shape[2:]
    # TODO Add support for dilation and padding size
    return conv2d_flops(
        in_channels=module.in_channels,
        out_channels=module.out_channels,
        input_shape=input_shape,
        kernel_shape=module.kernel_size,
        padding=module.padding_mode,
        strides=module.stride,
        dilation=module.dilation,
    )


def _linear_flops(module, activation):
    # Auxiliary func to use abstract flop computation
    return dense_flops(module.in_features, module.out_features)

In [18]:
def flops(model, input):
    """Compute Multiply-add FLOPs estimate from model
    Arguments:
        model {torch.nn.Module} -- Module to compute flops for
        input {torch.Tensor} -- Input tensor needed for activations
    Returns:
        tuple:
        - int - Number of total FLOPs
        - int - Number of FLOPs related to nonzero parameters
    """
    FLOP_fn = {
        nn.Conv2d: _conv2d_flops,
        nn.Linear: _linear_flops,
        Conv2dMasked: _conv2d_flops,
        LinearMasked: _linear_flops,
    }

    total_flops = nonzero_flops = 0
    activations = get_activations(model, input)

    # The ones we need for backprop
    for m, (act, _) in activations.items():
        if m.__class__ in FLOP_fn:
            w = m.weight.detach().cpu().numpy().copy()
            module_flops = FLOP_fn[m.__class__](m, act)
            total_flops += module_flops
            # For our operations, all weights are symmetric so we can just
            # do simple rule of three for the estimation
            nonzero_flops += module_flops * nonzero(w).sum() / np.prod(w.shape)

    return total_flops, nonzero_flops

In [19]:
train_loader, vali_loader = get_loaders()

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3019898.88it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2736316.94it/s]

Read  792  files
Read 633 images
Read 159 images


In [35]:
# net = EDAnet().cuda()
x, _ = next(iter(train_loader))
x = x.cuda()

# size, size_nz = model_size(net)


In [28]:
size, size_nz 

(681367, 679602)

In [91]:
net

EDAnet(
  (layers): ModuleList(
    (0): DownsampleBlock(
      (conv): Conv2d(3, 12, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (batchNorm): BatchNorm2d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (1): DownsampleBlock(
      (conv): Conv2d(15, 45, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (batchNorm): BatchNorm2d(60, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (2): EDABlock(
      (conv1x1_0): Conv2d(60, 40, kernel_size=(1, 1), stride=(1, 1))
      (batchNorm_0): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3x1_1): Conv2d(40, 40, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
      (conv1x3_1): Conv2d(40, 40, kernel_size=(1, 3),

In [38]:
best_model = torch.load('/home/user/Desktop/lanyard_segmentation/0425/backups_DeepLabv3plus_mobilenet_ghost/best_model.pth')
best_model_prune = best_model
x, _ = next(iter(train_loader))
x = x.cuda()
def prune_model_l1_unstructured(model, layer_type, proportion):
    for module in model.modules():       
        if isinstance(module, layer_type):
            mask = prune.l1_unstructured(module, 'weight', proportion)
            prune.remove(module, 'weight')
    return model, mask

# prune model
best_model_pruned, mask = prune_model_l1_unstructured(best_model_prune, nn.Conv2d, 0.2)

In [39]:
FLOPS = flops(best_model_pruned, x)

/home/user/anaconda3/envs/jamie_segmentation/lib/python3.6/site-packages/torch/nn/functional.py:3487: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


In [70]:
FLOPS
print('origin FLOPs = ' + str(FLOPS[0]/1000**3) + 'G')
print('pruned FLOPs = ' + str(FLOPS[1]/1000**3) + 'G')


origin FLOPs = 81.828095232G
pruned FLOPs = 65.4611551G


In [ ]:
# def prune_model_l1_unstructured(model, layer_type, proportion):
#     for module in model.modules():       
#         if isinstance(module, layer_type):
#             prune.l1_unstructured(module, 'weight', proportion)
#             prune.remove(module, 'weight')
#     return model

# # prune model
# best_model_pruned = prune_model_l1_unstructured(best_model_prune, nn.Conv2d, 0.2)
# # sparsified = best_model_pruned.to_sparse()
# def getGlobalSparsityRatio(pruned_model):
#     nparams = 0
#     pruned = 0
#     for k, v in dict(pruned_model.named_modules()).items():
#         if ((len(list(v.children())) == 0) and (k.endswith('conv'))):
#             nparams += v.weight.nelement()
#             pruned += torch.sum(v.weight == 0)
#     print('Global sparsity across the pruned layers: {:.2f}%'.format( 100. * pruned / float(nparams)))
    
# getGlobalSparsityRatio(best_model_pruned)

# Prune

In [9]:
# Experiment 1: Random weights pruning
# Change amount = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]
# amount = 0.7
# kwargs = {}
# pruning_strategy_1 = [
#     ("fc1", prune.random_unstructured, "weight", amount, kwargs),
#     ("fc2", prune.random_unstructured, "weight", amount, kwargs),
#     ("conv1", prune.random_unstructured, "weight", amount, kwargs),
#     ("conv2", prune.random_unstructured, "weight", amount, kwargs),
# ] #"fc1" etc change into layer type
# pruning_strategy_1 = (nn.Conv2d, prune.random_unstructured, 0.1)
# layer_type, prune_method, amount = pruning_strategy_1

In [54]:
# save_model = "aa"
# model_name= 'eda'
# a = f"/home/user/Desktop/lanyard_segmentation/prune/backups_prune/{save_model}_{model_name}_ghost_state_b.pth"
# a

'/home/user/Desktop/lanyard_segmentation/prune/backups_prune/aa_eda_ghost_state_b.pth'

In [20]:
class PruningExperiment:
    def __init__(
        self,
        pruning_strategy=None,
        batch_size_train=32,
        batch_size_test=32,
        epochs_prune_finetune=3,
        epochs_finetune=5,
        device="cuda",
        save_model=None,
        model_name='eda', 
        is_ghost = False,
        test_features_folder = '/home/user/Desktop/lanyard_segmentation/guardrail/test_data/features',
        test_labels_folder = '/home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels',
        orig_features_folder = '/home/user/Desktop/lanyard_segmentation/guardrail/orig_features',
        orig_labels_folder = '/home/user/Desktop/lanyard_segmentation/guardrail/orig_labels',
        viz_classes = ['guardrail'],
        classes = ['guardrail']
    ):

        """Initialize experiment
        :param pruning_strategy: Pruning strategy
        :param batch_size_train: Batch size for train
        :param batch_size_test: Batch size for test
        :param epochs_finetune: Number of epochs to finetune pruned model
        :param optimizer: Optimizer to perform gradient descent
        :param device: Device 'cpu' or 'cuda' where calculations are performed
        :param model_name: 'eda', 'unet', 'deeplab_mobile', 'deeplab_xception'
        :param is_ghost: True-> ghost module adopted
        :return: Outcome of pruning strategy: Accuracy and pruning metrics
        """
        self.pruning_strategy = pruning_strategy
        self.device = device
        self.batch_size_train = batch_size_train
        self.batch_size_test = batch_size_test
        self.epochs_prune_finetune = epochs_prune_finetune
        self.epochs_finetune = epochs_finetune
        self.save_model = save_model
        self.model_name = model_name
        self.is_ghost = is_ghost
        
        self.test_features_folder = test_features_folder
        self.test_labels_folder = test_labels_folder
        self.orig_features_folder = orig_features_folder
        self.orig_labels_folder = orig_labels_folder
        self.viz_classes = viz_classes
        self.classes = classes
        

    def load_model(self):

        """Load LeNet model.
        All experiments will be performed on a trained model
        from the original script.
        """
        if self.model_name == 'eda':
            if self.is_ghost:
                net = torch.load('/home/user/Desktop/lanyard_segmentation/0425/backups_EDAnet_ghost/best_model.pth').to(self.device)
            else:
                net = torch.load("/home/user/Desktop/lanyard_segmentation/0425/backups_EDAnet/best_model.pth").to(self.device)
        if self.model_name == 'unet':
            if self.is_ghost:
                net = torch.load('/home/user/Desktop/lanyard_segmentation/0425/backups_unetplusplus_ghost/best_model.pth').to(self.device)
            else:
                net = torch.load("/home/user/Desktop/lanyard_segmentation/0425/backups_unetplusplus/best_model.pth").to(self.device)
        if self.model_name == 'deeplab_mobile':
            if self.is_ghost:
                net = torch.load('/home/user/Desktop/lanyard_segmentation/0425/backups_DeepLabv3plus_mobilenet_ghost/best_model.pth').to(self.device)
            else:
                net = torch.load("/home/user/Desktop/lanyard_segmentation/0425/backups_DeepLabv3plus_mobilenet/best_model.pth").to(self.device)
        if self.model_name == 'deeplab_xception':
            if self.is_ghost:
                net = torch.load('/home/user/Desktop/lanyard_segmentation/0425/backups_DeepLabv3plus_xcetption_ghost/best_model.pth').to(self.device)
            else:
                net = torch.load("/home/user/Desktop/lanyard_segmentation/0425/backups_DeepLabv3plus_xcetption/best_model.pth").to(self.device)
        
        return net

    def prune_model(self, net, pruning_strategy):
        layer_type, prune_method, amount = pruning_strategy_1
        for module in net.modules():       
            if isinstance(module, layer_type):
                prune_method(module, 'weight', amount)
#                 module.set_mask(mask.weight_mask)
                prune.remove(module, 'weight')
            
#         for modulename, strategy, name, amount, kwargs in pruning_strategy:
            
#             module = getattr(net, modulename)
#             if kwargs:
#                 n = kwargs["n"]
#                 dim = kwargs["dim"]
#                 mask = strategy(module, name=name, amount=amount, n=n, dim=dim)
#             else:
#                 mask = strategy(module, name=name, amount=amount, n=n, dim=dim)
#             # print(modulename, mask.weight_mask.shape)
#             module.set_mask(mask.weight_mask)

        return net

    def train(self, net, optimizer, data_loader, device, mask=None):
#         scheduler.step()
        metrics = defaultdict(float)
        net.train()
        for data in data_loader:
           
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)  
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = calc_loss(outputs, labels, metrics)
            loss.backward()
            optimizer.step()
                


    def test(self, net):
        net.eval()
        iou_scores, iou_scores_resized = test_lanyard_re(net, self.classes, self.test_features_folder, self.test_labels_folder)
        return iou_scores, iou_scores_resized

    def calculate_prune_metrics(self, net, test_loader, device):

        x, _ = next(iter(test_loader))
        x = x.to(device)

        size, size_nz = model_size(net)

        FLOPS = flops(net, x)
        compression_ratio = size / size_nz
        print('original FLOPs = ' + str(FLOPS[0]/1000**3) + 'G')
        print('pruned FLOPs = ' + str(FLOPS[1]/1000**3) + 'G')
        print('original Params = ' + str(size))
        print('pruned Params = ' + str(size_nz))
        return FLOPS, compression_ratio

    def run(self):
        """
        Main function to run pruning -> finetuning -> evaluation
        """
        pruning_strategy = self.pruning_strategy
        batch_size_train = self.batch_size_train
        batch_size_test = self.batch_size_test
        epochs_prune_finetune = self.epochs_prune_finetune
        epochs_finetune = self.epochs_finetune
        device = self.device

        net = self.load_model()
       
        weight_decay = 1e-4
        max_lr = 5e-4
        # Observe that all parameters are being optimized
        optimizer_ft = torch.optim.Adam(net.parameters(), lr=max_lr, weight_decay=weight_decay)
#         lambda1 = lambda epoch: (1-epoch/epochs_prune_finetune*epochs_finetune) ** 0.9

#         exp_lr_scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer_ft, lr_lambda=[lambda1])
        
#         train_loader, test_loader = get_loaders(batch_size_train, batch_size_test)
        train_loader, vali_loader = get_loaders()

        print("Pruning cycle")
        print("=======================")
        for epoch in range(epochs_prune_finetune):
            if pruning_strategy is not None:
                net = self.prune_model(net, pruning_strategy)
            
            for finetune_epoch in range(epochs_finetune):
                self.train(net, optimizer_ft, train_loader, device)
#                 iou_scores, iou_scores_resized = self.test(net)
#                 print(
#                     f"\tAfter finetuning: Epoch {finetune_epoch}.\
#                         iou_scores {iou_scores:.4f} iou_scores_resized {iou_scores_resized:.4f}"
#                 )
            iou_scores, iou_scores_resized = self.test(net)
            print(f"After pruning: Epoch {epoch}.  iou_scores {iou_scores:.4f} iou_scores_resized {iou_scores_resized:.4f}.")

#         for modulename, strategy, name, amount, kwargs in pruning_strategy:
#             module = getattr(net, modulename)
#             prune.remove(module, name)
        if self.save_model is not None:
            if self.is_ghost:
                torch.save(net.state_dict(), f"/home/user/Desktop/lanyard_segmentation/prune/backups_prune/{self.save_model}_{self.model_name}_ghost_state_b.pth")
                torch.save(net, f"/home/user/Desktop/lanyard_segmentation/prune/backups_prune/{self.save_model}_{self.model_name}_ghost_whole_b.pth")
            else:
                torch.save(net.state_dict(), f"/home/user/Desktop/lanyard_segmentation/prune/backups_prune/{self.save_model}_{self.model_name}_state_b.pth")
                torch.save(net, f"/home/user/Desktop/lanyard_segmentation/prune/backups_prune/{self.save_model}_{self.model_name}_whole_b.pth")
                
        layer_type, prune_method, amount = pruning_strategy
        for module in net.modules():       
            if isinstance(module, layer_type):
                mask = prune_method(module, 'weight', amount)
                prune.remove(module, 'weight')
        iou_scores, iou_scores_resized = self.test(net)

        FLOPS, compression_ratio = self.calculate_prune_metrics(
            net, vali_loader, device
        )
        

        if self.save_model is not None:
            if self.is_ghost:
                torch.save(net.state_dict(), f"/home/user/Desktop/lanyard_segmentation/prune/backups_prune/{self.save_model}_{self.model_name}_ghost_state.pth")
                torch.save(net, f"/home/user/Desktop/lanyard_segmentation/prune/backups_prune/{self.save_model}_{self.model_name}_ghost_whole.pth")
            else:
                torch.save(net.state_dict(), f"/home/user/Desktop/lanyard_segmentation/prune/backups_prune/{self.save_model}_{self.model_name}_state.pth")
                torch.save(net, f"/home/user/Desktop/lanyard_segmentation/prune/backups_prune/{self.save_model}_{self.model_name}_whole.pth")
        print('iou score:{}, iou_resized_scored: {}'.format(iou_scores, iou_scores_resized))
        print('FLOPS[0] / FLOPS[1]: {}, compression_ratio: {}'.format(FLOPS[0] / FLOPS[1], compression_ratio))
        return iou_scores, iou_scores_resized, FLOPS[0] / FLOPS[1], compression_ratio

## Exp 0.1

In [25]:
pruning_strategy_1 = (nn.Conv2d, prune.random_unstructured, 0.1)
pruning_strategy_3 = (nn.Conv2d, prune.random_unstructured, 0.3)
pruning_strategy_5 = (nn.Conv2d, prune.random_unstructured, 0.5)
pruning_strategy_7 = (nn.Conv2d, prune.random_unstructured, 0.7)
# 'eda', 'unet', 'deeplab_mobile', 'deeplab_xception'

pe_eda_1 = PruningExperiment(
        pruning_strategy=pruning_strategy_1,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='eda', 
)
pe_eda_ghost_1 = PruningExperiment(
        pruning_strategy=pruning_strategy_1,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='eda',
        is_ghost = True,
)

pe_unet_1 = PruningExperiment(
        pruning_strategy=pruning_strategy_1,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='unet', 
)
pe_unet_ghost_1 = PruningExperiment(
        pruning_strategy=pruning_strategy_1,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='unet',
        is_ghost = True,
)

pe_deeplab_mobile_1 = PruningExperiment(
        pruning_strategy=pruning_strategy_1,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='deeplab_mobile', 
)
pe_deeplab_mobile_ghost_1 = PruningExperiment(
        pruning_strategy=pruning_strategy_1,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='deeplab_mobile',
        is_ghost = True,
)



In [32]:
print(pe_eda_1.run())
# original FLOPs = 4.383268864G
# pruned FLOPs = 3.94496G
# original Params = 681367
# pruned Params = 613862
# iou score:0.4657851678597035, iou_resized_scored: 0.43015158667748027
# FLOPS[0] / FLOPS[1]: 1.1111060350421804, compression_ratio: 1.1099677126129326
# (0.4657851678597035, 0.43015158667748027, 1.1111060350421804, 1.1099677126129326)
# After pruning: Epoch 0.  iou_scores 0.5539 iou_scores_resized 0.4643.
# After pruning: Epoch 1.  iou_scores 0.5759 iou_scores_resized 0.4942.
# After pruning: Epoch 2.  iou_scores 0.5189 iou_scores_resized 0.4793.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3151697.12it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3119144.38it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 282517.89it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 365945.99it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.383268864G
pruned FLOPs = 3.94496G
original Params = 681367
pruned Params = 613862
iou score:0.4657851678597035, iou_resized_scored: 0.43015158667748027
FLOPS[0] / FLOPS[1]: 1.1111060350421804, compression_ratio: 1.1099677126129326
(0.4657851678597035, 0.43015158667748027, 1.1111060350421804, 1.1099677126129326)


In [35]:
print(pe_eda_ghost_1.run())
# original FLOPs = 4.10015744G
# pruned FLOPs = 3.689586688G
# original Params = 443970
# pruned Params = 399029
# iou score:0.5280771692011291, iou_resized_scored: 0.4589798652550248
# FLOPS[0] / FLOPS[1]: 1.111278250578944, compression_ratio: 1.1126258993707225
# (0.5280771692011291, 0.4589798652550248, 1.111278250578944, 1.1126258993707225)
# After pruning: Epoch 0.  iou_scores 0.5167 iou_scores_resized 0.4387.
# After pruning: Epoch 1.  iou_scores 0.5435 iou_scores_resized 0.4773.
# After pruning: Epoch 2.  iou_scores 0.5794 iou_scores_resized 0.4858.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 2258252.05it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3305361.96it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 358723.37it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 136999.88it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.10015744G
pruned FLOPs = 3.689586688G
original Params = 443970
pruned Params = 399029
iou score:0.5280771692011291, iou_resized_scored: 0.4589798652550248
FLOPS[0] / FLOPS[1]: 1.111278250578944, compression_ratio: 1.1126258993707225
(0.5280771692011291, 0.4589798652550248, 1.111278250578944, 1.1126258993707225)


In [73]:
print(pe_deeplab_mobile_1.run())
# original FLOPs = 81.828095232G
# pruned FLOPs = 73.50169268G
# original Params = 5810913
# pruned Params = 5094426
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 1.1132817796217316, compression_ratio: 1.140641359792055
# (3.774281242139753e-12, 3.774281242139753e-12, 1.1132817796217316, 1.140641359792055)
# After pruning: Epoch 2.  iou_scores 0.5999 iou_scores_resized 0.4694.
# After pruning: Epoch 1.  iou_scores 0.5541 iou_scores_resized 0.4405.
# After pruning: Epoch 0.  iou_scores 0.5224 iou_scores_resized 0.4165.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3212658.38it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2765935.69it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
module: Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
module: Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
module: BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
module: BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
module: ReLU6(inplace=True)
module: ReLU6(inplace=True)
module: Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
module: Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
module: BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_runn

module: InvertedResidual(
  (conv): Sequential(
    (0): Conv2d(32, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
    (3): Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), groups=192, bias=False)
    (4): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU6(inplace=True)
    (6): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
module: InvertedResidual(
  (conv): Sequential(
    (0): Conv2d(32, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
    (3): Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), groups=192, bias=False)
    (4): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, tr

module: ReLU6(inplace=True)
module: Conv2d(960, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
module: Conv2d(960, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
module: BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
module: BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
module: InvertedResidual(
  (conv): Sequential(
    (0): Conv2d(160, 960, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(960, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU6(inplace=True)
    (3): Conv2d(960, 960, kernel_size=(3, 3), stride=(1, 1), groups=960, bias=False)
    (4): BatchNorm2d(960, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU6(inplace=True)
    (6): Conv2d(960, 160, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (7): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)
module: InvertedResidual(
  (conv): S

iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 1.1132817796217316, compression_ratio: 1.140641359792055
(3.774281242139753e-12, 3.774281242139753e-12, 1.1132817796217316, 1.140641359792055)


In [34]:
test_model = torch.load("/home/user/Desktop/lanyard_segmentation/0425/backups_DeepLabv3plus_mobilenet/best_model.pth").to('cuda')
test_model.eval()
# image = torch.randn(1, 3, 512, 512)
# with torch.no_grad():
#   output = test_model.forward(image)
# print(output.size())

DeepLabv3_plus_mobilenet(
  (efficient_features): MobileNetV2(
    (features): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
      )
      (1): InvertedResidual(
        (conv): Sequential(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
          (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (2): InvertedResidual(
        (conv): Sequential(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=

In [78]:
print(pe_deeplab_mobile_ghost_1.run())
# original FLOPs = 76.213077248G
# pruned FLOPs = 68.469927688G
# original Params = 5128001
# pruned Params = 4499743
# iou score:1.5572264902721547e-05, iou_resized_scored: 3.768811365102572e-12
# FLOPS[0] / FLOPS[1]: 1.1130883268240557, compression_ratio: 1.1396208627914972
# (1.5572264902721547e-05, 3.768811365102572e-12, 1.1130883268240557, 1.1396208627914972)
# After pruning: Epoch 2.  iou_scores 0.5892 iou_scores_resized 0.4913.
# After pruning: Epoch 1.  iou_scores 0.4937 iou_scores_resized 0.3996.
# After pruning: Epoch 0.  iou_scores 0.5534 iou_scores_resized 0.4435.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 2698528.65it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2417677.41it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 242337.56it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 400926.12it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 76.213077248G
pruned FLOPs = 68.469927688G
original Params = 5128001
pruned Params = 4499743
iou score:1.5572264902721547e-05, iou_resized_scored: 3.768811365102572e-12
FLOPS[0] / FLOPS[1]: 1.1130883268240557, compression_ratio: 1.1396208627914972
(1.5572264902721547e-05, 3.768811365102572e-12, 1.1130883268240557, 1.1396208627914972)


In [36]:
# test_features_folder = '/home/user/Desktop/lanyard_segmentation/guardrail/test_data/features'
# test_labels_folder = '/home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels'

# classes = ['guardrail']

# iou_scores, iou_scores_resized = test_lanyard_re(test_model, classes, test_features_folder, test_labels_folder)

path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/features
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 250119.05it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 376041.05it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)


In [37]:
# iou_scores, iou_scores_resized

(0.5696587830507065, 0.44248591706144247)

In [26]:
print(pe_unet_1.run())
original FLOPs = 195.780411392G
pruned FLOPs = 176.12502016G
original Params = 8596703
pruned Params = 7727227
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 1.1115990857752303, compression_ratio: 1.1125210893895054
(3.774281242139753e-12, 3.774281242139753e-12, 1.1115990857752303, 1.1125210893895054)
After pruning: Epoch 0.  iou_scores 0.5477 iou_scores_resized 0.5112.

After pruning: Epoch 1.  iou_scores 0.5602 iou_scores_resized 0.5084.

After pruning: Epoch 2.  iou_scores 0.4921 iou_scores_resized 0.4648.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 2881083.06it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2960685.18it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 356378.77it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 272629.76it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 195.780411392G
pruned FLOPs = 176.12502016G
original Params = 8596703
pruned Params = 7727227
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 1.1115990857752303, compression_ratio: 1.1125210893895054
(3.774281242139753e-12, 3.774281242139753e-12, 1.1115990857752303, 1.1125210893895054)


In [40]:
print(pe_unet_ghost_1.run())
# original FLOPs = 87.09472256G
# pruned FLOPs = 78.116291584G
# original Params = 4599921
# pruned Params = 4097398
# iou score:0.001783559214772855, iou_resized_scored: 0.003125641540495625
# FLOPS[0] / FLOPS[1]: 1.1149367282283915, compression_ratio: 1.122644419702455
# (0.001783559214772855, 0.003125641540495625, 1.1149367282283915, 1.122644419702455)
# After pruning: Epoch 0.  iou_scores 0.6571 iou_scores_resized 0.6007.
# After pruning: Epoch 1.  iou_scores 0.7071 iou_scores_resized 0.6585.
# After pruning: Epoch 2.  iou_scores 0.6941 iou_scores_resized 0.6357.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 1857879.62it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 1764146.98it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 351780.34it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 416228.64it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 87.09472256G
pruned FLOPs = 78.116291584G
original Params = 4599921
pruned Params = 4097398
iou score:0.001783559214772855, iou_resized_scored: 0.003125641540495625
FLOPS[0] / FLOPS[1]: 1.1149367282283915, compression_ratio: 1.122644419702455
(0.001783559214772855, 0.003125641540495625, 1.1149367282283915, 1.122644419702455)


## Exp 0.3

In [57]:
pruning_strategy_1 = (nn.Conv2d, prune.random_unstructured, 0.1)
pruning_strategy_3 = (nn.Conv2d, prune.random_unstructured, 0.3)
pruning_strategy_5 = (nn.Conv2d, prune.random_unstructured, 0.5)
pruning_strategy_7 = (nn.Conv2d, prune.random_unstructured, 0.7)
# 'eda', 'unet', 'deeplab_mobile', 'deeplab_xception'

pe_eda_3 = PruningExperiment(
        pruning_strategy=pruning_strategy_3,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='eda', 
)
pe_eda_ghost_3 = PruningExperiment(
        pruning_strategy=pruning_strategy_3,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='eda',
        is_ghost = True,
)

pe_unet_3 = PruningExperiment(
        pruning_strategy=pruning_strategy_3,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='unet', 
)
pe_unet_ghost_3 = PruningExperiment(
        pruning_strategy=pruning_strategy_3,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='unet',
        is_ghost = True,
)

pe_deeplab_mobile_3 = PruningExperiment(
        pruning_strategy=pruning_strategy_3,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='deeplab_mobile', 
)
pe_deeplab_mobile_ghost_3 = PruningExperiment(
        pruning_strategy=pruning_strategy_3,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='deeplab_mobile',
        is_ghost = True,
)



In [43]:
print(pe_eda_3.run())
# original FLOPs = 4.383268864G
# pruned FLOPs = 3.068309504G
# original Params = 681367
# pruned Params = 478853
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 1.4285615118962913, compression_ratio: 1.4229147567207472
# (3.774281242139753e-12, 3.774281242139753e-12, 1.4285615118962913, 1.4229147567207472)
# After pruning: Epoch 0.  iou_scores 0.5711 iou_scores_resized 0.5007.
# After pruning: Epoch 1.  iou_scores 0.5305 iou_scores_resized 0.4667.
# After pruning: Epoch 2.  iou_scores 0.6074 iou_scores_resized 0.5003.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3116218.36it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3113297.81it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 406910.09it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 403895.94it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.383268864G
pruned FLOPs = 3.068309504G
original Params = 681367
pruned Params = 478853
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 1.4285615118962913, compression_ratio: 1.4229147567207472
(3.774281242139753e-12, 3.774281242139753e-12, 1.4285615118962913, 1.4229147567207472)


In [46]:
print(pe_eda_ghost_3.run())
# original FLOPs = 4.10015744G
# pruned FLOPs = 2.869305344G
# original Params = 443970
# pruned Params = 311638
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 1.4289721547320968, compression_ratio: 1.4246337096246287
# (3.774281242139753e-12, 3.774281242139753e-12, 1.4289721547320968, 1.4246337096246287)
# After pruning: Epoch 0.  iou_scores 0.5504 iou_scores_resized 0.4425.
# After pruning: Epoch 1.  iou_scores 0.6043 iou_scores_resized 0.4963.
# After pruning: Epoch 2.  iou_scores 0.5912 iou_scores_resized 0.4948.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3269575.56it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3348678.19it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 269930.46it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 381300.36it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.10015744G
pruned FLOPs = 2.869305344G
original Params = 443970
pruned Params = 311638
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 1.4289721547320968, compression_ratio: 1.4246337096246287
(3.774281242139753e-12, 3.774281242139753e-12, 1.4289721547320968, 1.4246337096246287)


In [49]:
print(pe_deeplab_mobile_3.run())
# original FLOPs = 81.828095232G
# pruned FLOPs = 57.165742692G
# original Params = 5810913
# pruned Params = 3965818
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 1.4314183876325524, compression_ratio: 1.4652495399435879
# (3.774281242139753e-12, 3.774281242139753e-12, 1.4314183876325524, 1.4652495399435879
# After pruning: Epoch 0.  iou_scores 0.5034 iou_scores_resized 0.4255.
# After pruning: Epoch 1.  iou_scores 0.5532 iou_scores_resized 0.4179.
# After pruning: Epoch 2.  iou_scores 0.5626 iou_scores_resized 0.4780.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 2736316.94it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2834376.08it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 181149.34it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 378652.44it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 81.828095232G
pruned FLOPs = 57.165742692G
original Params = 5810913
pruned Params = 3965818
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 1.4314183876325524, compression_ratio: 1.4652495399435879
(3.774281242139753e-12, 3.774281242139753e-12, 1.4314183876325524, 1.4652495399435879)


In [52]:
print(pe_deeplab_mobile_ghost_3.run())
# original FLOPs = 76.213077248G
# pruned FLOPs = 53.249522276G
# original Params = 5128001
# pruned Params = 3503118
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 1.4312443377984982, compression_ratio: 1.4638390713644245
# (3.774281242139753e-12, 3.774281242139753e-12, 1.4312443377984982, 1.4638390713644245)
# After pruning: Epoch 0.  iou_scores 0.4760 iou_scores_resized 0.3883.
# After pruning: Epoch 1.  iou_scores 0.5626 iou_scores_resized 0.4572.
# After pruning: Epoch 2.  iou_scores 0.5669 iou_scores_resized 0.4996.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3575768.32it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3642421.89it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 354064.62it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 413075.39it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 76.213077248G
pruned FLOPs = 53.249522276G
original Params = 5128001
pruned Params = 3503118
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 1.4312443377984982, compression_ratio: 1.4638390713644245
(3.774281242139753e-12, 3.774281242139753e-12, 1.4312443377984982, 1.4638390713644245)


In [55]:
print(pe_unet_3.run())
# original FLOPs = 195.780411392G
# pruned FLOPs = 136.98145792G
# original Params = 8596703
# pruned Params = 6012115
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 1.4292475373297588, compression_ratio: 1.4298966337137595
# (3.774281242139753e-12, 3.774281242139753e-12, 1.4292475373297588, 1.4298966337137595)
# After pruning: Epoch 0.  iou_scores 0.5112 iou_scores_resized 0.4714.
# After pruning: Epoch 1.  iou_scores 0.5681 iou_scores_resized 0.4981.
# After pruning: Epoch 2.  iou_scores 0.5316 iou_scores_resized 0.4648.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3614677.66it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3295524.57it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 381300.36it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 416228.64it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 195.780411392G
pruned FLOPs = 136.98145792G
original Params = 8596703
pruned Params = 6012115
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 1.4292475373297588, compression_ratio: 1.4298966337137595
(3.774281242139753e-12, 3.774281242139753e-12, 1.4292475373297588, 1.4298966337137595)


In [58]:
print(pe_unet_ghost_3.run())
# original FLOPs = 87.09472256G
# pruned FLOPs = 60.747357184G
# original Params = 4599921
# pruned Params = 3188380
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 1.4337203558698932, compression_ratio: 1.4427141683237255
# (3.774281242139753e-12, 3.774281242139753e-12, 1.4337203558698932, 1.4427141683237255)
# After pruning: Epoch 0.  iou_scores 0.7115 iou_scores_resized 0.6522.
# After pruning: Epoch 1.  iou_scores 0.6991 iou_scores_resized 0.6261.
# After pruning: Epoch 2.  iou_scores 0.6976 iou_scores_resized 0.6128.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3493048.13it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3145728.00it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 296336.70it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 155344.59it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 87.09472256G
pruned FLOPs = 60.747357184G
original Params = 4599921
pruned Params = 3188380
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 1.4337203558698932, compression_ratio: 1.4427141683237255
(3.774281242139753e-12, 3.774281242139753e-12, 1.4337203558698932, 1.4427141683237255)


## Exp 0.5

In [59]:
pruning_strategy_1 = (nn.Conv2d, prune.random_unstructured, 0.1)
pruning_strategy_3 = (nn.Conv2d, prune.random_unstructured, 0.3)
pruning_strategy_5 = (nn.Conv2d, prune.random_unstructured, 0.5)
pruning_strategy_7 = (nn.Conv2d, prune.random_unstructured, 0.7)
# 'eda', 'unet', 'deeplab_mobile', 'deeplab_xception'


In [61]:
pe_eda_5 = PruningExperiment(
        pruning_strategy=pruning_strategy_5,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='eda', 
)
print(pe_eda_5.run())
# original FLOPs = 4.383268864G
# pruned FLOPs = 2.19162624G
# original Params = 681367
# pruned Params = 343842
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 2.000007475727248, compression_ratio: 1.9816281896917771
# (3.774281242139753e-12, 3.774281242139753e-12, 2.000007475727248, 1.9816281896917771)
# After pruning: Epoch 0.  iou_scores 0.5223 iou_scores_resized 0.4654.
# After pruning: Epoch 1.  iou_scores 0.5418 iou_scores_resized 0.4905.
# After pruning: Epoch 2.  iou_scores 0.5675 iou_scores_resized 0.4887.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3393144.81it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3471148.14it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 271273.39it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 227191.47it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.383268864G
pruned FLOPs = 2.19162624G
original Params = 681367
pruned Params = 343842
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 2.000007475727248, compression_ratio: 1.9816281896917771
(3.774281242139753e-12, 3.774281242139753e-12, 2.000007475727248, 1.9816281896917771)


In [63]:
pe_eda_ghost_5 = PruningExperiment(
        pruning_strategy=pruning_strategy_5,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='eda',
        is_ghost = True,
)
print(pe_eda_ghost_5.run())
# original FLOPs = 4.10015744G
# pruned FLOPs = 2.049654784G
# original Params = 443970
# pruned Params = 224252
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 2.0004136657580673, compression_ratio: 1.979781674187967
# (3.774281242139753e-12, 3.774281242139753e-12, 2.0004136657580673, 1.979781674187967)
# After pruning: Epoch 0.  iou_scores 0.5956 iou_scores_resized 0.4999.
# After pruning: Epoch 1.  iou_scores 0.6002 iou_scores_resized 0.5018.
# After pruning: Epoch 2.  iou_scores 0.6201 iou_scores_resized 0.5075.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3113297.81it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3130903.65it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 207323.01it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 172006.16it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.10015744G
pruned FLOPs = 2.049654784G
original Params = 443970
pruned Params = 224252
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 2.0004136657580673, compression_ratio: 1.979781674187967
(3.774281242139753e-12, 3.774281242139753e-12, 2.0004136657580673, 1.979781674187967)


In [65]:
pe_deeplab_mobile_5 = PruningExperiment(
        pruning_strategy=pruning_strategy_5,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='deeplab_mobile', 
)
print(pe_deeplab_mobile_5.run())
# original FLOPs = 81.828095232G
# pruned FLOPs = 40.831586944G
# original Params = 5810913
# pruned Params = 2842801
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 2.0040390628026823, compression_ratio: 2.0440801167580847
# (3.774281242139753e-12, 3.774281242139753e-12, 2.0040390628026823, 2.0440801167580847)
# After pruning: Epoch 0.  iou_scores 0.4942 iou_scores_resized 0.4187.
# After pruning: Epoch 1.  iou_scores 0.5610 iou_scores_resized 0.4824.
# After pruning: Epoch 2.  iou_scores 0.5887 iou_scores_resized 0.5115.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3407065.40it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3006234.18it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 330460.32it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 238104.59it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 81.828095232G
pruned FLOPs = 40.831586944G
original Params = 5810913
pruned Params = 2842801
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 2.0040390628026823, compression_ratio: 2.0440801167580847
(3.774281242139753e-12, 3.774281242139753e-12, 2.0040390628026823, 2.0440801167580847)


In [67]:
pe_deeplab_mobile_ghost_5 = PruningExperiment(
        pruning_strategy=pruning_strategy_5,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='deeplab_mobile',
        is_ghost = True,
)
print(pe_deeplab_mobile_ghost_5.run())
# original FLOPs = 76.213077248G
# pruned FLOPs = 38.035433088G
# original Params = 5128001
# pruned Params = 2512371
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 2.003738910285864, compression_ratio: 2.04110021967297
# (3.774281242139753e-12, 3.774281242139753e-12, 2.003738910285864, 2.04110021967297)
# After pruning: Epoch 0.  iou_scores 0.5226 iou_scores_resized 0.4484.
# After pruning: Epoch 1.  iou_scores 0.5760 iou_scores_resized 0.4814.
# After pruning: Epoch 2.  iou_scores 0.5722 iou_scores_resized 0.4800.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3703331.96it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3292258.44it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 361099.02it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 416228.64it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 76.213077248G
pruned FLOPs = 38.035433088G
original Params = 5128001
pruned Params = 2512371
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 2.003738910285864, compression_ratio: 2.04110021967297
(3.774281242139753e-12, 3.774281242139753e-12, 2.003738910285864, 2.04110021967297)


In [69]:
pe_unet_5 = PruningExperiment(
        pruning_strategy=pruning_strategy_5,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='unet', 
)
print(pe_unet_5.run())
# original FLOPs = 195.780411392G
# pruned FLOPs = 97.846299648G
# original Params = 8596703
# pruned Params = 4299150
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 2.0008974493293654, compression_ratio: 1.999628531221288
# (3.774281242139753e-12, 3.774281242139753e-12, 2.0008974493293654, 1.999628531221288)
# After pruning: Epoch 0.  iou_scores 0.4637 iou_scores_resized 0.4255.
# After pruning: Epoch 1.  iou_scores 0.5186 iou_scores_resized 0.5003.
# After pruning: Epoch 2.  iou_scores 0.4864 iou_scores_resized 0.4646.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3438808.25it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3127955.53it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 276781.48it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 419430.40it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 195.780411392G
pruned FLOPs = 97.846299648G
original Params = 8596703
pruned Params = 4299150
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 2.0008974493293654, compression_ratio: 1.999628531221288
(3.774281242139753e-12, 3.774281242139753e-12, 2.0008974493293654, 1.999628531221288)


In [71]:
pe_unet_ghost_5 = PruningExperiment(
        pruning_strategy=pruning_strategy_5,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='unet',
        is_ghost = True,
)

print(pe_unet_ghost_5.run())
# original FLOPs = 87.09472256G
# pruned FLOPs = 43.387926528G
# original Params = 4599921
# pruned Params = 2274843
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 2.007349268091763, compression_ratio: 2.0220828426401294
# (3.774281242139753e-12, 3.774281242139753e-12, 2.007349268091763, 2.0220828426401294)
# After pruning: Epoch 0.  iou_scores 0.7035 iou_scores_resized 0.6094.
# After pruning: Epoch 1.  iou_scores 0.6433 iou_scores_resized 0.5858.
# After pruning: Epoch 2.  iou_scores 0.6981 iou_scores_resized 0.6449.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3393144.81it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3348678.19it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 356378.77it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 251271.67it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 87.09472256G
pruned FLOPs = 43.387926528G
original Params = 4599921
pruned Params = 2274843
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 2.007349268091763, compression_ratio: 2.0220828426401294
(3.774281242139753e-12, 3.774281242139753e-12, 2.007349268091763, 2.0220828426401294)


## Exp 0.7

In [72]:
pruning_strategy_1 = (nn.Conv2d, prune.random_unstructured, 0.1)
pruning_strategy_3 = (nn.Conv2d, prune.random_unstructured, 0.3)
pruning_strategy_5 = (nn.Conv2d, prune.random_unstructured, 0.5)
pruning_strategy_7 = (nn.Conv2d, prune.random_unstructured, 0.7)
# 'eda', 'unet', 'deeplab_mobile', 'deeplab_xception'


In [74]:
pe_eda_7 = PruningExperiment(
        pruning_strategy=pruning_strategy_7,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp7",
        model_name='eda', 
)

print(pe_eda_7.run())
# original FLOPs = 4.383268864G
# pruned FLOPs = 1.314975744G
# original Params = 681367
# pruned Params = 208833
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 3.3333457928787467, compression_ratio: 3.262736253369918
# (3.774281242139753e-12, 3.774281242139753e-12, 3.3333457928787467, 3.262736253369918)
# After pruning: Epoch 0.  iou_scores 0.5659 iou_scores_resized 0.4769.
# After pruning: Epoch 1.  iou_scores 0.5662 iou_scores_resized 0.4571.
# After pruning: Epoch 2.  iou_scores 0.6107 iou_scores_resized 0.5114.

path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3341940.41it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3453106.83it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 365945.99it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 395115.59it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.383268864G
pruned FLOPs = 1.314975744G
original Params = 681367
pruned Params = 208833
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 3.3333457928787467, compression_ratio: 3.262736253369918
(3.774281242139753e-12, 3.774281242139753e-12, 3.3333457928787467, 3.262736253369918)


In [76]:
pe_eda_ghost_7 = PruningExperiment(
        pruning_strategy=pruning_strategy_7,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp7",
        model_name='eda',
        is_ghost = True,
)
print(pe_eda_ghost_7.run())

# original FLOPs = 4.10015744G
# pruned FLOPs = 1.229815808G
# original Params = 443970
# pruned Params = 136863
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 3.333960592576803, compression_ratio: 3.243900835141711
# (3.774281242139753e-12, 3.774281242139753e-12, 3.333960592576803, 3.243900835141711)
# After pruning: Epoch 0.  iou_scores 0.5475 iou_scores_resized 0.4772.

# After pruning: Epoch 1.  iou_scores 0.5844 iou_scores_resized 0.5121.

# After pruning: Epoch 2.  iou_scores 0.6114 iou_scores_resized 0.5036.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3266360.64it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3365642.12it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 376041.05it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 25829.44it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.10015744G
pruned FLOPs = 1.229815808G
original Params = 443970
pruned Params = 136863
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 3.333960592576803, compression_ratio: 3.243900835141711
(3.774281242139753e-12, 3.774281242139753e-12, 3.333960592576803, 3.243900835141711)


In [78]:
pe_unet_7 = PruningExperiment(
        pruning_strategy=pruning_strategy_7,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp7",
        model_name='unet', 
)
print(pe_unet_7.run())
# original FLOPs = 195.780411392G
# pruned FLOPs = 58.707013632G
# original Params = 8596703
# pruned Params = 2586375
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 3.3348726034547274, compression_ratio: 3.323842443574501
# (3.774281242139753e-12, 3.774281242139753e-12, 3.3348726034547274, 3.323842443574501)
# After pruning: Epoch 0.  iou_scores 0.5834 iou_scores_resized 0.5469.

# After pruning: Epoch 1.  iou_scores 0.5191 iou_scores_resized 0.4916.

# After pruning: Epoch 2.  iou_scores 0.5282 iou_scores_resized 0.5235.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3253563.93it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3369055.55it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 365945.99it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 406910.09it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 195.780411392G
pruned FLOPs = 58.707013632G
original Params = 8596703
pruned Params = 2586375
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 3.3348726034547274, compression_ratio: 3.323842443574501
(3.774281242139753e-12, 3.774281242139753e-12, 3.3348726034547274, 3.323842443574501)


In [80]:
pe_unet_ghost_7 = PruningExperiment(
        pruning_strategy=pruning_strategy_7,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp7",
        model_name='unet',
        is_ghost = True,
)
print(pe_unet_ghost_7.run())

# original FLOPs = 87.09472256G
# pruned FLOPs = 26.015314944G
# original Params = 4599921
# pruned Params = 1369170
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 3.3478250310433757, compression_ratio: 3.359641972874077
# (3.774281242139753e-12, 3.774281242139753e-12, 3.3478250310433757, 3.359641972874077)
# After pruning: Epoch 0.  iou_scores 0.6784 iou_scores_resized 0.6290.

# After pruning: Epoch 1.  iou_scores 0.6934 iou_scores_resized 0.6482.

# After pruning: Epoch 2.  iou_scores 0.6301 iou_scores_resized 0.6072.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3308654.15it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2898681.30it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 354064.62it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 413075.39it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 87.09472256G
pruned FLOPs = 26.015314944G
original Params = 4599921
pruned Params = 1369170
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 3.3478250310433757, compression_ratio: 3.359641972874077
(3.774281242139753e-12, 3.774281242139753e-12, 3.3478250310433757, 3.359641972874077)


In [82]:
pe_deeplab_mobile_7 = PruningExperiment(
        pruning_strategy=pruning_strategy_7,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp7",
        model_name='deeplab_mobile', 
)
print(pe_deeplab_mobile_7.run())
# original FLOPs = 81.828095232G
# pruned FLOPs = 24.497808028G
# original Params = 5810913
# pruned Params = 1720152
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 3.340221098086564, compression_ratio: 3.3781392574609685
# (3.774281242139753e-12, 3.774281242139753e-12, 3.340221098086564, 3.3781392574609685)
# After pruning: Epoch 0.  iou_scores 0.5064 iou_scores_resized 0.4221.

# After pruning: Epoch 1.  iou_scores 0.5051 iou_scores_resized 0.4618.

# After pruning: Epoch 2.  iou_scores 0.5865 iou_scores_resized 0.4722.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3575768.32it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3064473.03it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 349525.33it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 403895.94it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 81.828095232G
pruned FLOPs = 24.497808028G
original Params = 5810913
pruned Params = 1720152
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 3.340221098086564, compression_ratio: 3.3781392574609685
(3.774281242139753e-12, 3.774281242139753e-12, 3.340221098086564, 3.3781392574609685)


In [84]:
pe_deeplab_mobile_ghost_7 = PruningExperiment(
        pruning_strategy=pruning_strategy_7,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp7",
        model_name='deeplab_mobile',
        is_ghost = True,
)
print(pe_deeplab_mobile_ghost_7.run())
# original FLOPs = 76.213077248G
# pruned FLOPs = 22.821585692G
# original Params = 5128001
# pruned Params = 1521854
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 3.3395171692524475, compression_ratio: 3.3695748738052402
# (3.774281242139753e-12, 3.774281242139753e-12, 3.3395171692524475, 3.3695748738052402)
# After pruning: Epoch 0.  iou_scores 0.5496 iou_scores_resized 0.4189.

# After pruning: Epoch 1.  iou_scores 0.5220 iou_scores_resized 0.4336.

# After pruning: Epoch 2.  iou_scores 0.5577 iou_scores_resized 0.4546.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3017156.01it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3070137.49it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 336579.95it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 392273.04it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 76.213077248G
pruned FLOPs = 22.821585692G
original Params = 5128001
pruned Params = 1521854
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 3.3395171692524475, compression_ratio: 3.3695748738052402
(3.774281242139753e-12, 3.774281242139753e-12, 3.3395171692524475, 3.3695748738052402)


# prune_l1

## Exp 0.1

In [89]:
pruning_strategy_1 = (nn.Conv2d, prune.l1_unstructured, 0.1)
pruning_strategy_3 = (nn.Conv2d, prune.l1_unstructured, 0.3)
pruning_strategy_5 = (nn.Conv2d, prune.l1_unstructured, 0.5)
pruning_strategy_7 = (nn.Conv2d, prune.l1_unstructured, 0.7)
# 'eda', 'unet', 'deeplab_mobile', 'deeplab_xception'

# pe_eda_1 = PruningExperiment(
#         pruning_strategy=pruning_strategy_1,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp1",
#         model_name='eda', 
# )
# print(pe_eda_1.run())
# pe_eda_ghost_1 = PruningExperiment(
#         pruning_strategy=pruning_strategy_1,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp1",
#         model_name='eda',
#         is_ghost = True,
# )
# print(pe_eda_ghost_1.run())
# pe_unet_1 = PruningExperiment(
#         pruning_strategy=pruning_strategy_1,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp1",
#         model_name='unet', 
# )
# print(pe_unet_1.run())
# pe_unet_ghost_1 = PruningExperiment(
#         pruning_strategy=pruning_strategy_1,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp1",
#         model_name='unet',
#         is_ghost = True,
# )
# print(pe_unet_ghost_1.run())
# pe_deeplab_mobile_1 = PruningExperiment(
#         pruning_strategy=pruning_strategy_1,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp1",
#         model_name='deeplab_mobile', 
# )
# print(pe_deeplab_mobile_1.run())
# pe_deeplab_mobile_ghost_1 = PruningExperiment(
#         pruning_strategy=pruning_strategy_1,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp1",
#         model_name='deeplab_mobile',
#         is_ghost = True,
# )
# print(pe_deeplab_mobile_ghost_1.run())


In [90]:
pe_eda_1 = PruningExperiment(
        pruning_strategy=pruning_strategy_1,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='eda', 
)
print(pe_eda_1.run())

# original FLOPs = 4.383268864G
# pruned FLOPs = 3.94496G
# original Params = 681367
# pruned Params = 613862
# iou score:0.5559587970410736, iou_resized_scored: 0.5139585491179616
# FLOPS[0] / FLOPS[1]: 1.1111060350421804, compression_ratio: 1.1099677126129326
# (0.5559587970410736, 0.5139585491179616, 1.1111060350421804, 1.1099677126129326)
# After pruning: Epoch 0.  iou_scores 0.5609 iou_scores_resized 0.4427.

# After pruning: Epoch 1.  iou_scores 0.5244 iou_scores_resized 0.3989.

# After pruning: Epoch 2.  iou_scores 0.5559 iou_scores_resized 0.5138.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3456700.07it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3424627.60it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 242337.56it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 238104.59it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.383268864G
pruned FLOPs = 3.94496G
original Params = 681367
pruned Params = 613862
iou score:0.5559587970410736, iou_resized_scored: 0.5139585491179616
FLOPS[0] / FLOPS[1]: 1.1111060350421804, compression_ratio: 1.1099677126129326
(0.5559587970410736, 0.5139585491179616, 1.1111060350421804, 1.1099677126129326)


In [92]:
pe_eda_ghost_1 = PruningExperiment(
        pruning_strategy=pruning_strategy_1,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='eda',
        is_ghost = True,
)

print(pe_eda_ghost_1.run())
# original FLOPs = 4.10015744G
# pruned FLOPs = 3.690340352G
# original Params = 443970
# pruned Params = 399031
# iou score:0.6250240527691192, iou_resized_scored: 0.502096567681112
# FLOPS[0] / FLOPS[1]: 1.1110512985009358, compression_ratio: 1.1126203227318179
# (0.6250240527691192, 0.502096567681112, 1.1110512985009358, 1.1126203227318179)
# After pruning: Epoch 2.  iou_scores 0.6261 iou_scores_resized 0.5031.
# After pruning: Epoch 1.  iou_scores 0.6029 iou_scores_resized 0.4877.
# After pruning: Epoch 0.  iou_scores 0.5891 iou_scores_resized 0.5068.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3345305.91it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3033688.37it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 334515.04it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 30008.78it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.10015744G
pruned FLOPs = 3.690340352G
original Params = 443970
pruned Params = 399031
iou score:0.6250240527691192, iou_resized_scored: 0.502096567681112
FLOPS[0] / FLOPS[1]: 1.1110512985009358, compression_ratio: 1.1126203227318179
(0.6250240527691192, 0.502096567681112, 1.1110512985009358, 1.1126203227318179)


In [94]:
pe_unet_1 = PruningExperiment(
        pruning_strategy=pruning_strategy_1,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='unet', 
)
print(pe_unet_1.run())
# original FLOPs = 195.780411392G
# pruned FLOPs = 176.202738688G
# original Params = 8596703
# pruned Params = 7738660
# iou score:0.5145729142610123, iou_resized_scored: 0.47036385068133235
# FLOPS[0] / FLOPS[1]: 1.1111087878075832, compression_ratio: 1.11087746457397
# (0.5145729142610123, 0.47036385068133235, 1.1111087878075832, 1.11087746457397)
# After pruning: Epoch 0.  iou_scores 0.4911 iou_scores_resized 0.4299.

# After pruning: Epoch 1.  iou_scores 0.5660 iou_scores_resized 0.5025.

# After pruning: Epoch 2.  iou_scores 0.5156 iou_scores_resized 0.4720.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3308654.15it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2198470.40it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 358723.37it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 392273.04it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 195.780411392G
pruned FLOPs = 176.202738688G
original Params = 8596703
pruned Params = 7738660
iou score:0.5145729142610123, iou_resized_scored: 0.47036385068133235
FLOPS[0] / FLOPS[1]: 1.1111087878075832, compression_ratio: 1.11087746457397
(0.5145729142610123, 0.47036385068133235, 1.1111087878075832, 1.11087746457397)


In [96]:
pe_unet_ghost_1 = PruningExperiment(
        pruning_strategy=pruning_strategy_1,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='unet',
        is_ghost = True,
)
print(pe_unet_ghost_1.run())

# original FLOPs = 87.09472256G
# pruned FLOPs = 78.396724224G
# original Params = 4599921
# pruned Params = 4140660
# iou score:0.7019023282147194, iou_resized_scored: 0.6267605329728523
# FLOPS[0] / FLOPS[1]: 1.1109484920715251, compression_ratio: 1.1109149266059033
# (0.7019023282147194, 0.6267605329728523, 1.1109484920715251, 1.1109149266059033)
# After pruning: Epoch 0.  iou_scores 0.6924 iou_scores_resized 0.6439.

# After pruning: Epoch 1.  iou_scores 0.7138 iou_scores_resized 0.6172.

# After pruning: Epoch 2.  iou_scores 0.7020 iou_scores_resized 0.6251.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3295524.57it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2630157.38it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 320740.89it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 18763.23it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 87.09472256G
pruned FLOPs = 78.396724224G
original Params = 4599921
pruned Params = 4140660
iou score:0.7019023282147194, iou_resized_scored: 0.6267605329728523
FLOPS[0] / FLOPS[1]: 1.1109484920715251, compression_ratio: 1.1109149266059033
(0.7019023282147194, 0.6267605329728523, 1.1109484920715251, 1.1109149266059033)


In [98]:
pe_deeplab_mobile_1 = PruningExperiment(
        pruning_strategy=pruning_strategy_1,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='deeplab_mobile', 
)
print(pe_deeplab_mobile_1.run())
# original FLOPs = 81.828095232G
# pruned FLOPs = 73.64399284G
# original Params = 5810913
# pruned Params = 5233391
# iou score:0.5390870777820934, iou_resized_scored: 0.4746534358642615
# FLOPS[0] / FLOPS[1]: 1.1111306173985012, compression_ratio: 1.1103533062979625
# (0.5390870777820934, 0.4746534358642615, 1.1111306173985012, 1.1103533062979625)
# After pruning: Epoch 0.  iou_scores 0.5344 iou_scores_resized 0.4410.

# After pruning: Epoch 1.  iou_scores 0.5424 iou_scores_resized 0.4992.

# After pruning: Epoch 2.  iou_scores 0.5369 iou_scores_resized 0.4741.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3522681.62it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3019898.88it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 258416.83it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 406910.09it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 81.828095232G
pruned FLOPs = 73.64399284G
original Params = 5810913
pruned Params = 5233391
iou score:0.5390870777820934, iou_resized_scored: 0.4746534358642615
FLOPS[0] / FLOPS[1]: 1.1111306173985012, compression_ratio: 1.1103533062979625
(0.5390870777820934, 0.4746534358642615, 1.1111306173985012, 1.1103533062979625)


In [100]:
pe_deeplab_mobile_ghost_1 = PruningExperiment(
        pruning_strategy=pruning_strategy_1,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp1",
        model_name='deeplab_mobile',
        is_ghost = True,
)
print(pe_deeplab_mobile_ghost_1.run())
# original FLOPs = 76.213077248G
# pruned FLOPs = 68.591721864G
# original Params = 5128001
# pruned Params = 4618771
# iou score:0.5573610254314912, iou_resized_scored: 0.47435463047653365
# FLOPS[0] / FLOPS[1]: 1.11111188313819, compression_ratio: 1.110252272736622
# (0.5573610254314912, 0.47435463047653365, 1.11111188313819, 1.110252272736622)
# After pruning: Epoch 0.  iou_scores 0.5547 iou_scores_resized 0.4384.

# After pruning: Epoch 1.  iou_scores 0.5401 iou_scores_resized 0.4475.

# After pruning: Epoch 2.  iou_scores 0.5579 iou_scores_resized 0.4747.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3335229.69it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 1931330.68it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 84799.30it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 232025.33it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 76.213077248G
pruned FLOPs = 68.591721864G
original Params = 5128001
pruned Params = 4618771
iou score:0.5573610254314912, iou_resized_scored: 0.47435463047653365
FLOPS[0] / FLOPS[1]: 1.11111188313819, compression_ratio: 1.110252272736622
(0.5573610254314912, 0.47435463047653365, 1.11111188313819, 1.110252272736622)


## EXP 0.3

In [102]:
pe_eda_3 = PruningExperiment(
        pruning_strategy=pruning_strategy_3,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp3",
        model_name='eda', 
)
print(pe_eda_3.run())
# original FLOPs = 4.383268864G
# pruned FLOPs = 3.068309504G
# original Params = 681367
# pruned Params = 478853
# iou score:0.6129338716095514, iou_resized_scored: 0.49395028290178833
# FLOPS[0] / FLOPS[1]: 1.4285615118962913, compression_ratio: 1.4229147567207472
# (0.6129338716095514, 0.49395028290178833, 1.4285615118962913, 1.4229147567207472)
# After pruning: Epoch 0.  iou_scores 0.5634 iou_scores_resized 0.4878.

# After pruning: Epoch 1.  iou_scores 0.5451 iou_scores_resized 0.4556.

# After pruning: Epoch 2.  iou_scores 0.6365 iou_scores_resized 0.5328.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 2327882.81it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3244032.00it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 386708.88it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 383985.58it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.383268864G
pruned FLOPs = 3.068309504G
original Params = 681367
pruned Params = 478853
iou score:0.6129338716095514, iou_resized_scored: 0.49395028290178833
FLOPS[0] / FLOPS[1]: 1.4285615118962913, compression_ratio: 1.4229147567207472
(0.6129338716095514, 0.49395028290178833, 1.4285615118962913, 1.4229147567207472)


In [104]:
pe_eda_ghost_3 = PruningExperiment(
        pruning_strategy=pruning_strategy_3,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp3",
        model_name='eda',
        is_ghost = True,
)
print(pe_eda_ghost_3.run())
# original FLOPs = 4.10015744G
# pruned FLOPs = 2.870059008G
# original Params = 443970
# pruned Params = 311640
# iou score:0.574646283744664, iou_resized_scored: 0.4494632499798665
# FLOPS[0] / FLOPS[1]: 1.428596913363532, compression_ratio: 1.4246245668078552
# (0.574646283744664, 0.4494632499798665, 1.428596913363532, 1.4246245668078552)
# After pruning: Epoch 0.  iou_scores 0.5693 iou_scores_resized 0.4317.

# After pruning: Epoch 1.  iou_scores 0.5899 iou_scores_resized 0.4964.

# After pruning: Epoch 2.  iou_scores 0.6201 iou_scores_resized 0.5170.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3122075.91it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 1149044.89it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 338670.51it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 237069.36it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.10015744G
pruned FLOPs = 2.870059008G
original Params = 443970
pruned Params = 311640
iou score:0.574646283744664, iou_resized_scored: 0.4494632499798665
FLOPS[0] / FLOPS[1]: 1.428596913363532, compression_ratio: 1.4246245668078552
(0.574646283744664, 0.4494632499798665, 1.428596913363532, 1.4246245668078552)


In [106]:
pe_unet_3 = PruningExperiment(
        pruning_strategy=pruning_strategy_3,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp3",
        model_name='unet', 
)
print(pe_unet_3.run())
# original FLOPs = 195.780411392G
# pruned FLOPs = 137.04526336G
# original Params = 8596703
# pruned Params = 6022567
# iou score:0.34667043383856055, iou_resized_scored: 0.2613014109287641
# FLOPS[0] / FLOPS[1]: 1.428582109238686, compression_ratio: 1.4274150872875304
# (0.34667043383856055, 0.2613014109287641, 1.428582109238686, 1.4274150872875304)
# After pruning: Epoch 0.  iou_scores 0.5204 iou_scores_resized 0.5002.

# After pruning: Epoch 1.  iou_scores 0.5160 iou_scores_resized 0.4236.

# After pruning: Epoch 2.  iou_scores 0.5323 iou_scores_resized 0.4910.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3442371.78it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3386227.08it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 351780.34it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 392273.04it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 195.780411392G
pruned FLOPs = 137.04526336G
original Params = 8596703
pruned Params = 6022567
iou score:0.34667043383856055, iou_resized_scored: 0.2613014109287641
FLOPS[0] / FLOPS[1]: 1.428582109238686, compression_ratio: 1.4274150872875304
(0.34667043383856055, 0.2613014109287641, 1.428582109238686, 1.4274150872875304)


In [108]:
pe_unet_ghost_3 = PruningExperiment(
        pruning_strategy=pruning_strategy_3,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp3",
        model_name='unet',
        is_ghost = True,
)
print(pe_unet_ghost_3.run())
# original FLOPs = 87.09472256G
# pruned FLOPs = 60.979690496G
# original Params = 4599921
# pruned Params = 3222137
# iou score:0.5969300567226651, iou_resized_scored: 0.45411785354218465
# FLOPS[0] / FLOPS[1]: 1.4282578650626807, compression_ratio: 1.4275994471991724
# (0.5969300567226651, 0.45411785354218465, 1.4282578650626807, 1.4275994471991724)
# After pruning: Epoch 0.  iou_scores 0.6880 iou_scores_resized 0.6286.

# After pruning: Epoch 1.  iou_scores 0.6846 iou_scores_resized 0.6399.

# After pruning: Epoch 2.  iou_scores 0.7011 iou_scores_resized 0.6063.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3075822.93it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 1825213.61it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 209715.20it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 245612.40it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 87.09472256G
pruned FLOPs = 60.979690496G
original Params = 4599921
pruned Params = 3222137
iou score:0.5969300567226651, iou_resized_scored: 0.45411785354218465
FLOPS[0] / FLOPS[1]: 1.4282578650626807, compression_ratio: 1.4275994471991724
(0.5969300567226651, 0.45411785354218465, 1.4282578650626807, 1.4275994471991724)


In [110]:
pe_deeplab_mobile_3 = PruningExperiment(
        pruning_strategy=pruning_strategy_3,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp3",
        model_name='deeplab_mobile', 
)
print(pe_deeplab_mobile_3.run())
# original FLOPs = 81.828095232G
# pruned FLOPs = 57.280987748G
# original Params = 5810913
# pruned Params = 4078362
# iou score:0.41660745691801826, iou_resized_scored: 0.3578706157152785
# FLOPS[0] / FLOPS[1]: 1.428538481074937, compression_ratio: 1.424815403831244
# (0.41660745691801826, 0.3578706157152785, 1.428538481074937, 1.424815403831244)
# After pruning: Epoch 0.  iou_scores 0.5277 iou_scores_resized 0.4398.

# After pruning: Epoch 1.  iou_scores 0.5810 iou_scores_resized 0.4785.

# After pruning: Epoch 2.  iou_scores 0.5418 iou_scores_resized 0.4853.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3352057.28it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3200278.20it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 328469.59it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 406910.09it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 81.828095232G
pruned FLOPs = 57.280987748G
original Params = 5810913
pruned Params = 4078362
iou score:0.41660745691801826, iou_resized_scored: 0.3578706157152785
FLOPS[0] / FLOPS[1]: 1.428538481074937, compression_ratio: 1.424815403831244
(0.41660745691801826, 0.3578706157152785, 1.428538481074937, 1.424815403831244)


In [112]:
pe_deeplab_mobile_ghost_3 = PruningExperiment(
        pruning_strategy=pruning_strategy_3,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp3",
        model_name='deeplab_mobile',
        is_ghost = True,
)
print(pe_deeplab_mobile_ghost_3.run())
# original FLOPs = 76.213077248G
# pruned FLOPs = 53.34896778G
# original Params = 5128001
# pruned Params = 3600322
# iou score:0.11715321553774785, iou_resized_scored: 0.08939222775431256
# FLOPS[0] / FLOPS[1]: 1.4285764171162, compression_ratio: 1.4243173249503795
# (0.11715321553774785, 0.08939222775431256, 1.4285764171162, 1.4243173249503795)
# After pruning: Epoch 0.  iou_scores 0.5583 iou_scores_resized 0.4445.

# After pruning: Epoch 1.  iou_scores 0.6040 iou_scores_resized 0.4899.

# After pruning: Epoch 2.  iou_scores 0.5758 iou_scores_resized 0.4989.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 1885294.42it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 1893893.25it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 342930.52it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 409969.56it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 76.213077248G
pruned FLOPs = 53.34896778G
original Params = 5128001
pruned Params = 3600322
iou score:0.11715321553774785, iou_resized_scored: 0.08939222775431256
FLOPS[0] / FLOPS[1]: 1.4285764171162, compression_ratio: 1.4243173249503795
(0.11715321553774785, 0.08939222775431256, 1.4285764171162, 1.4243173249503795)


## Exp 0.5

In [113]:
pruning_strategy_1 = (nn.Conv2d, prune.l1_unstructured, 0.1)
pruning_strategy_3 = (nn.Conv2d, prune.l1_unstructured, 0.3)
pruning_strategy_5 = (nn.Conv2d, prune.l1_unstructured, 0.5)
pruning_strategy_7 = (nn.Conv2d, prune.l1_unstructured, 0.7)
# 'eda', 'unet', 'deeplab_mobile', 'deeplab_xception'

# pe_eda_5 = PruningExperiment(
#         pruning_strategy=pruning_strategy_5,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp5",
#         model_name='eda', 
# )
# print(pe_eda_5.run())
# pe_eda_ghost_5 = PruningExperiment(
#         pruning_strategy=pruning_strategy_5,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp5",
#         model_name='eda',
#         is_ghost = True,
# )
# print(pe_eda_ghost_5.run())
# pe_unet_5 = PruningExperiment(
#         pruning_strategy=pruning_strategy_5,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp5",
#         model_name='unet', 
# )
# print(pe_unet_5.run())
# pe_unet_ghost_5 = PruningExperiment(
#         pruning_strategy=pruning_strategy_5,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp5",
#         model_name='unet',
#         is_ghost = True,
# )
# print(pe_unet_ghost_5.run())
# pe_deeplab_mobile_5 = PruningExperiment(
#         pruning_strategy=pruning_strategy_5,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp5",
#         model_name='deeplab_mobile', 
# )
# print(pe_deeplab_mobile_5.run())
# pe_deeplab_mobile_ghost_5 = PruningExperiment(
#         pruning_strategy=pruning_strategy_5,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp5",
#         model_name='deeplab_mobile',
#         is_ghost = True,
# )
# print(pe_deeplab_mobile_ghost_5.run())


In [115]:
pe_eda_5 = PruningExperiment(
        pruning_strategy=pruning_strategy_5,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp5",
        model_name='eda', 
)
print(pe_eda_5.run())
# original FLOPs = 4.383268864G
# pruned FLOPs = 2.19162624G
# original Params = 681367
# pruned Params = 343842
# iou score:0.07094982354929322, iou_resized_scored: 0.024525577858636554
# FLOPS[0] / FLOPS[1]: 2.000007475727248, compression_ratio: 1.9816281896917771
# (0.07094982354929322, 0.024525577858636554, 2.000007475727248, 1.9816281896917771)
# After pruning: Epoch 0.  iou_scores 0.5483 iou_scores_resized 0.4425.

# After pruning: Epoch 1.  iou_scores 0.5854 iou_scores_resized 0.4584.

# After pruning: Epoch 2.  iou_scores 0.5832 iou_scores_resized 0.4564.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3428161.78it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3410563.42it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 334515.04it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 386708.88it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.383268864G
pruned FLOPs = 2.19162624G
original Params = 681367
pruned Params = 343842
iou score:0.07094982354929322, iou_resized_scored: 0.024525577858636554
FLOPS[0] / FLOPS[1]: 2.000007475727248, compression_ratio: 1.9816281896917771
(0.07094982354929322, 0.024525577858636554, 2.000007475727248, 1.9816281896917771)


In [117]:
pe_eda_ghost_5 = PruningExperiment(
        pruning_strategy=pruning_strategy_5,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp5",
        model_name='eda',
        is_ghost = True,
)
print(pe_eda_ghost_5.run())
# original FLOPs = 4.10015744G
# pruned FLOPs = 2.050031616G
# original Params = 443970
# pruned Params = 224253
# iou score:0.28142085574568826, iou_resized_scored: 0.15495145305775979
# FLOPS[0] / FLOPS[1]: 2.0000459544132223, compression_ratio: 1.979772845848216
# (0.28142085574568826, 0.15495145305775979, 2.0000459544132223, 1.979772845848216)
# After pruning: Epoch 0.  iou_scores 0.5741 iou_scores_resized 0.4742.

# After pruning: Epoch 1.  iou_scores 0.5827 iou_scores_resized 0.5234.

# After pruning: Epoch 2.  iou_scores 0.6168 iou_scores_resized 0.5261.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3311952.91it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2808020.94it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 356378.77it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 409969.56it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.10015744G
pruned FLOPs = 2.050031616G
original Params = 443970
pruned Params = 224253
iou score:0.28142085574568826, iou_resized_scored: 0.15495145305775979
FLOPS[0] / FLOPS[1]: 2.0000459544132223, compression_ratio: 1.979772845848216
(0.28142085574568826, 0.15495145305775979, 2.0000459544132223, 1.979772845848216)


In [119]:
pe_unet_5 = PruningExperiment(
        pruning_strategy=pruning_strategy_5,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp5",
        model_name='unet', 
)
print(pe_unet_5.run())
# original FLOPs = 195.780411392G
# pruned FLOPs = 97.890074624G
# original Params = 8596703
# pruned Params = 4306481
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 2.0000026779425903, compression_ratio: 1.9962245276363695
# (3.774281242139753e-12, 3.774281242139753e-12, 2.0000026779425903, 1.9962245276363695)
# After pruning: Epoch 0.  iou_scores 0.5251 iou_scores_resized 0.4963.

# After pruning: Epoch 1.  iou_scores 0.5795 iou_scores_resized 0.5285.

# After pruning: Epoch 2.  iou_scores 0.5693 iou_scores_resized 0.5285.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3269575.56it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2981946.83it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 340787.20it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 255990.38it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 195.780411392G
pruned FLOPs = 97.890074624G
original Params = 8596703
pruned Params = 4306481
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 2.0000026779425903, compression_ratio: 1.9962245276363695
(3.774281242139753e-12, 3.774281242139753e-12, 2.0000026779425903, 1.9962245276363695)


In [121]:
pe_unet_ghost_5 = PruningExperiment(
        pruning_strategy=pruning_strategy_5,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp5",
        model_name='unet',
        is_ghost = True,
)
print(pe_unet_ghost_5.run())
# original FLOPs = 87.09472256G
# pruned FLOPs = 43.54736128G
# original Params = 4599921
# pruned Params = 2303609
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 2.0, compression_ratio: 1.996832361741945
# (3.774281242139753e-12, 3.774281242139753e-12, 2.0, 1.996832361741945)
# After pruning: Epoch 0.  iou_scores 0.7104 iou_scores_resized 0.6158.

# After pruning: Epoch 1.  iou_scores 0.7170 iou_scores_resized 0.6585.

# After pruning: Epoch 2.  iou_scores 0.7083 iou_scores_resized 0.6718.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3160693.40it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2965972.11it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 392273.04it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 413075.39it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 87.09472256G
pruned FLOPs = 43.54736128G
original Params = 4599921
pruned Params = 2303609
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 2.0, compression_ratio: 1.996832361741945
(3.774281242139753e-12, 3.774281242139753e-12, 2.0, 1.996832361741945)


In [123]:
pe_deeplab_mobile_5 = PruningExperiment(
        pruning_strategy=pruning_strategy_5,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp5",
        model_name='deeplab_mobile', 
)
print(pe_deeplab_mobile_5.run())
# original FLOPs = 81.828095232G
# pruned FLOPs = 40.914047616G
# original Params = 5810913
# pruned Params = 2923329
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 2.0, compression_ratio: 1.9877725018292502
# (3.774281242139753e-12, 3.774281242139753e-12, 2.0, 1.9877725018292502)
# After pruning: Epoch 0.  iou_scores 0.5171 iou_scores_resized 0.4355.

# After pruning: Epoch 1.  iou_scores 0.5710 iou_scores_resized 0.4653.

# After pruning: Epoch 2.  iou_scores 0.5831 iou_scores_resized 0.4676.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3386227.08it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2157070.63it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 347299.06it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 403895.94it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 81.828095232G
pruned FLOPs = 40.914047616G
original Params = 5810913
pruned Params = 2923329
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 2.0, compression_ratio: 1.9877725018292502
(3.774281242139753e-12, 3.774281242139753e-12, 2.0, 1.9877725018292502)


In [125]:
pe_deeplab_mobile_ghost_5 = PruningExperiment(
        pruning_strategy=pruning_strategy_5,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp5",
        model_name='deeplab_mobile',
        is_ghost = True,
)
print(pe_deeplab_mobile_ghost_5.run())
# original FLOPs = 76.213077248G
# pruned FLOPs = 38.106538624G
# original Params = 5128001
# pruned Params = 2581873
# iou score:0.046893076194745004, iou_resized_scored: 0.04261902138026904
# FLOPS[0] / FLOPS[1]: 2.0, compression_ratio: 1.9861553995878185
# (0.046893076194745004, 0.04261902138026904, 2.0, 1.9861553995878185)
# After pruning: Epoch 0.  iou_scores 0.5453 iou_scores_resized 0.4526.

# After pruning: Epoch 1.  iou_scores 0.5758 iou_scores_resized 0.4779.

# After pruning: Epoch 2.  iou_scores 0.5663 iou_scores_resized 0.4640.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3442371.78it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2488306.19it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 326502.71it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 253609.08it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 76.213077248G
pruned FLOPs = 38.106538624G
original Params = 5128001
pruned Params = 2581873
iou score:0.046893076194745004, iou_resized_scored: 0.04261902138026904
FLOPS[0] / FLOPS[1]: 2.0, compression_ratio: 1.9861553995878185
(0.046893076194745004, 0.04261902138026904, 2.0, 1.9861553995878185)


## exp 0.7

In [126]:
pruning_strategy_1 = (nn.Conv2d, prune.l1_unstructured, 0.1)
pruning_strategy_3 = (nn.Conv2d, prune.l1_unstructured, 0.3)
pruning_strategy_5 = (nn.Conv2d, prune.l1_unstructured, 0.5)
pruning_strategy_7 = (nn.Conv2d, prune.l1_unstructured, 0.7)
# 'eda', 'unet', 'deeplab_mobile', 'deeplab_xception'

# pe_eda_7 = PruningExperiment(
#         pruning_strategy=pruning_strategy_7,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp7",
#         model_name='eda', 
# )
# print(pe_eda_7.run())
# pe_eda_ghost_7 = PruningExperiment(
#         pruning_strategy=pruning_strategy_7,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp7",
#         model_name='eda',
#         is_ghost = True,
# )
# print(pe_eda_ghost_7.run())
# pe_unet_7 = PruningExperiment(
#         pruning_strategy=pruning_strategy_7,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp7",
#         model_name='unet', 
# )
# print(pe_unet_7.run())
# pe_unet_ghost_7 = PruningExperiment(
#         pruning_strategy=pruning_strategy_7,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp7",
#         model_name='unet',
#         is_ghost = True,
# )
# print(pe_unet_ghost_7.run())
# pe_deeplab_mobile_7 = PruningExperiment(
#         pruning_strategy=pruning_strategy_7,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp7",
#         model_name='deeplab_mobile', 
# )
# print(pe_deeplab_mobile_7.run())
# pe_deeplab_mobile_ghost_7 = PruningExperiment(
#         pruning_strategy=pruning_strategy_7,
#         epochs_prune_finetune=3,
#         epochs_finetune=4,
#         save_model="exp7",
#         model_name='deeplab_mobile',
#         is_ghost = True,
# )
# print(pe_deeplab_mobile_ghost_7.run())


In [128]:
pe_eda_7 = PruningExperiment(
        pruning_strategy=pruning_strategy_7,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp7",
        model_name='eda', 
)
print(pe_eda_7.run())
# original FLOPs = 4.383268864G
# pruned FLOPs = 1.314975744G
# original Params = 681367
# pruned Params = 208833
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 3.3333457928787467, compression_ratio: 3.262736253369918
# (3.774281242139753e-12, 3.774281242139753e-12, 3.3333457928787467, 3.262736253369918)
# After pruning: Epoch 0.  iou_scores 0.5870 iou_scores_resized 0.4863.

# After pruning: Epoch 1.  iou_scores 0.6212 iou_scores_resized 0.5012.

# After pruning: Epoch 2.  iou_scores 0.5887 iou_scores_resized 0.5017.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3081529.47it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 1350361.29it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 365945.99it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 201202.77it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.383268864G
pruned FLOPs = 1.314975744G
original Params = 681367
pruned Params = 208833
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 3.3333457928787467, compression_ratio: 3.262736253369918
(3.774281242139753e-12, 3.774281242139753e-12, 3.3333457928787467, 3.262736253369918)


In [130]:
pe_eda_ghost_7 = PruningExperiment(
        pruning_strategy=pruning_strategy_7,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp7",
        model_name='eda',
        is_ghost = True,
)
print(pe_eda_ghost_7.run())
# original FLOPs = 4.10015744G
# pruned FLOPs = 1.229815808G
# original Params = 443970
# pruned Params = 136863
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 3.333960592576803, compression_ratio: 3.243900835141711
# (3.774281242139753e-12, 3.774281242139753e-12, 3.333960592576803, 3.243900835141711)
# After pruning: Epoch 0.  iou_scores 0.5841 iou_scores_resized 0.4513.

# After pruning: Epoch 1.  iou_scores 0.5872 iou_scores_resized 0.4976.

# After pruning: Epoch 2.  iou_scores 0.6335 iou_scores_resized 0.5124.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3298797.19it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2815159.97it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 204217.05it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 184833.74it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 4.10015744G
pruned FLOPs = 1.229815808G
original Params = 443970
pruned Params = 136863
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 3.333960592576803, compression_ratio: 3.243900835141711
(3.774281242139753e-12, 3.774281242139753e-12, 3.333960592576803, 3.243900835141711)


In [132]:
pe_unet_7 = PruningExperiment(
        pruning_strategy=pruning_strategy_7,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp7",
        model_name='unet', 
)
print(pe_unet_7.run())
# original FLOPs = 195.780411392G
# pruned FLOPs = 58.735148032G
# original Params = 8596703
# pruned Params = 2590396
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 3.333275184482981, compression_ratio: 3.318682934964384
# (3.774281242139753e-12, 3.774281242139753e-12, 3.333275184482981, 3.318682934964384)
# After pruning: Epoch 0.  iou_scores 0.4985 iou_scores_resized 0.4651.

# After pruning: Epoch 1.  iou_scores 0.5449 iou_scores_resized 0.4399.

# After pruning: Epoch 2.  iou_scores 0.5636 iou_scores_resized 0.5237.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3474779.05it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3365642.12it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 338670.51it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 251271.67it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 195.780411392G
pruned FLOPs = 58.735148032G
original Params = 8596703
pruned Params = 2590396
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 3.333275184482981, compression_ratio: 3.318682934964384
(3.774281242139753e-12, 3.774281242139753e-12, 3.333275184482981, 3.318682934964384)


In [134]:
pe_unet_ghost_7 = PruningExperiment(
        pruning_strategy=pruning_strategy_7,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp7",
        model_name='unet',
        is_ghost = True,
)
print(pe_unet_ghost_7.run())
# original FLOPs = 87.09472256G
# pruned FLOPs = 26.115032064G
# original Params = 4599921
# pruned Params = 1385081
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 3.3350417624055497, compression_ratio: 3.3210483719002717
# (3.774281242139753e-12, 3.774281242139753e-12, 3.3350417624055497, 3.3210483719002717)
# After pruning: Epoch 0.  iou_scores 0.6991 iou_scores_resized 0.6132.

# After pruning: Epoch 1.  iou_scores 0.7165 iou_scores_resized 0.6601.

# After pruning: Epoch 2.  iou_scores 0.7076 iou_scores_resized 0.6538.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3298797.19it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3365642.12it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 392273.04it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 409969.56it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 87.09472256G
pruned FLOPs = 26.115032064G
original Params = 4599921
pruned Params = 1385081
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 3.3350417624055497, compression_ratio: 3.3210483719002717
(3.774281242139753e-12, 3.774281242139753e-12, 3.3350417624055497, 3.3210483719002717)


In [136]:
pe_deeplab_mobile_7 = PruningExperiment(
        pruning_strategy=pruning_strategy_7,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp7",
        model_name='deeplab_mobile', 
)
print(pe_deeplab_mobile_7.run())
# original FLOPs = 81.828095232G
# pruned FLOPs = 24.547107484G
# original Params = 5810913
# pruned Params = 1768296
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 3.333512727938972, compression_ratio: 3.2861653252622864
# (3.774281242139753e-12, 3.774281242139753e-12, 3.333512727938972, 3.2861653252622864)
# After pruning: Epoch 0.  iou_scores 0.4353 iou_scores_resized 0.3833.

# After pruning: Epoch 1.  iou_scores 0.5437 iou_scores_resized 0.4526.

# After pruning: Epoch 2.  iou_scores 0.5533 iou_scores_resized 0.4940.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3421100.69it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 3358835.96it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 347299.06it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 345100.96it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 81.828095232G
pruned FLOPs = 24.547107484G
original Params = 5810913
pruned Params = 1768296
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 3.333512727938972, compression_ratio: 3.2861653252622864
(3.774281242139753e-12, 3.774281242139753e-12, 3.333512727938972, 3.2861653252622864)


In [138]:
pe_deeplab_mobile_ghost_7 = PruningExperiment(
        pruning_strategy=pruning_strategy_7,
        epochs_prune_finetune=3,
        epochs_finetune=4,
        save_model="exp7",
        model_name='deeplab_mobile',
        is_ghost = True,
)
print(pe_deeplab_mobile_ghost_7.run())
# original FLOPs = 76.213077248G
# pruned FLOPs = 22.864109468G
# original Params = 5128001
# pruned Params = 1563424
# iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
# FLOPS[0] / FLOPS[1]: 3.3333061737946013, compression_ratio: 3.2799809904414925
# (3.774281242139753e-12, 3.774281242139753e-12, 3.3333061737946013, 3.2799809904414925)
# After pruning: Epoch 0.  iou_scores 0.5695 iou_scores_resized 0.4596.

# After pruning: Epoch 1.  iou_scores 0.6145 iou_scores_resized 0.5032.

# After pruning: Epoch 2.  iou_scores 0.4525 iou_scores_resized 0.4054.


path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/features_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_7.png', '0006_5

100%|██████████| 792/792 [00:00<00:00, 3511510.33it/s]


Read  792  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/train_data/labels_cropped
directory： []
file： ['0001_4.png', '0079_7.png', '0048_9.png', '0087_7.png', '0010_10.png', '0047_9.png', '0042_1.png', '0051_7.png', '0060_6.png', '0039_7.png', '0002_8.png', '0028_8.png', '0011_10.png', '0024_1.png', '0013_6.png', '0058_3.png', '0017_9.png', '0065_10.png', '0046_5.png', '0026_5.png', '0056_9.png', '0043_10.png', '0036_1.png', '0032_7.png', '0057_1.png', '0074_6.png', '0025_8.png', '0020_2.png', '0079_6.png', '0064_4.png', '0047_1.png', '0066_4.png', '0080_5.png', '0071_9.png', '0065_9.png', '0051_8.png', '0031_6.png', '0044_9.png', '0020_7.png', '0019_6.png', '0016_4.png', '0017_10.png', '0059_9.png', '0015_4.png', '0045_9.png', '0051_4.png', '0029_9.png', '0082_10.png', '0030_8.png', '0023_2.png', '0079_9.png', '0019_3.png', '0013_4.png', '0002_3.png', '0047_6.png', '0002_1.png', '0057_7.png', '0040_3.png', '0010_1.png', '0012_4.png', '0079_4.png', '0050_8.png', '0067_

100%|██████████| 792/792 [00:00<00:00, 2971277.97it/s]

Read  792  files
Read 633 images
Read 159 images
Pruning cycle


...

100%|██████████| 13/13 [00:00<00:00, 347299.06it/s]


Read  13  files
path： /home/user/Desktop/lanyard_segmentation/guardrail/test_data/labels
directory： []
file： ['0075.png', '0086.png', '0019.png', '0083.png', '0054.png', '0001.png', '0050.png', '0056.png', '0058.png', '0084.png', '0081.png', '0034.png', '0023.png']


100%|██████████| 13/13 [00:00<00:00, 400926.12it/s]

Read  13  files
Read 13 images
Read 13 images
Read 13 images


(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
(720, 960, 3)
original FLOPs = 76.213077248G
pruned FLOPs = 22.864109468G
original Params = 5128001
pruned Params = 1563424
iou score:3.774281242139753e-12, iou_resized_scored: 3.774281242139753e-12
FLOPS[0] / FLOPS[1]: 3.3333061737946013, compression_ratio: 3.2799809904414925
(3.774281242139753e-12, 3.774281242139753e-12, 3.3333061737946013, 3.2799809904414925)
